# Reward Competition Elo rating Calculation

## Importing other Python Libraries/Modules

In [1]:
import copy
import re
import os
import sys
import string
import glob
import ast
from collections import Counter
from collections import defaultdict
import warnings

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import git
# Getting the path of the root directory so that we can import repo specific functions
git_repo_object = git.Repo('.', search_parent_directories=True)
git_repo_directory = git_repo_object.working_tree_dir

In [4]:
# Setting path so that we can import functions
sys.path.append(os.path.join(git_repo_directory, "src"))

In [5]:
os.path.join(git_repo_directory, "src")


'/nancy/projects/dominance_strain_comparison/src'

In [6]:
from elorating import calculation
from elorating import dataframe

In [7]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

# Summary

TODO: Add summary

- Experiment
    - Observing competitve behavior between two mice inside a tube. Mice are categorized as winner and loser based on if they pushed the other mouse out of the tube or were pushed.
- Data
    - Excel spreadsheet of recorded tube test results. The relevant columns are those of the "Date" of the recording, the "winner" of the interaction, and the "loser" of the interaction. Each row will be for one interaction between two mice.
    - There is a cage for each sheet of the spreadsheet
    - For each recording session, we will assume that the date will be specified for that first row. We will use this to seperate all the rows into seperate sessions.
- Purpose of this Jupyter Notebook
    - To calculate the Elo rating of each mice after each interaction. The mice start off with an Elo rating of 1000. Elo ratings are calculated with the formula from here: https://www.omnicalculator.com/sports/elo . Then we will plot the change of Elo rating across all interactions. With the number of interactions on the X-Axis and the current Elo rating on the Y. There will be a line for mice.

## Name of protocol for naming

- This name will be used to name files and title plots. Please change if you are using a different protocol or adding more details
    - **NOTE**: This should be changed based on the name the protocol

In [8]:
protocol_name = "reward_competition"

In [9]:
prefix_name = "rc"

In [10]:
cohort_name = ""

In [11]:
if not cohort_name:
    cohort_name = input("""Type out the name of the cohort you are using. 
                        Make sure that this is typed as one word with individual words separated by underscore. i.e. pilot_3""")

Type out the name of the cohort you are using. 
                        Make sure that this is typed as one word with individual words separated by underscore. i.e. pilot_3pilot_1


## Getting the file name of the raw data

- Default input folder and keyword to search the files for 
    - **NOTE**: This should not be changed unless there is a consistent change with the file naming convention

In [12]:
input_folder = os.path.join(git_repo_directory, "data")

In [13]:
# Accomodates for both capitalization of the file names
raw_data_file_keyword = 'pilot_1/*[rR]eward*'

# NOTE: If there is a set excel file used for this notebook, make the cell below into a code cell and put the path in the "" (quotation marks)

In [14]:
raw_data_file_path = ""

- Asking the user what the path to the recording files are, with the option of using wildcards

In [15]:
if not raw_data_file_path:
    raw_data_glob_pattern = input("""Type out the path(address) of the raw behavioral recording excel sheets.
    Remember that if you are using a relative path, it will be based off of the location of this Jupyter Notebook.

    Globbing can also be used if you want to search with a wild card(Capitalization matters). 
    i.e. './data/*Home*' will be able to find './data/Homecage_observations.xlsx'

    NOTE: If left blank, the path will automatically be guessed off of the default settings 
    """)

Type out the path(address) of the raw behavioral recording excel sheets.
    Remember that if you are using a relative path, it will be based off of the location of this Jupyter Notebook.

    Globbing can also be used if you want to search with a wild card(Capitalization matters). 
    i.e. './data/*Home*' will be able to find './data/Homecage_observations.xlsx'

    NOTE: If left blank, the path will automatically be guessed off of the default settings 
    


- By default, this will search for files that are in the `./data` folder (in the folder that this notebook is in) that have key word you specified in the file name

In [16]:
if not raw_data_file_path:
    # Using the user inputted path/pattern
    if raw_data_glob_pattern.strip():
        # Getting a list of all matching files
        raw_data_files_list = glob.glob(raw_data_glob_pattern.strip(), recursive=True)
        # Checking if there were any files that matched
        if not raw_data_files_list:
            raise ValueError("No files were found with the path/pattern of {}. Please rerun the previous cell with the correct path".format(raw_data_glob_pattern))

    # Using the default pattern
    else:
        raw_data_glob_pattern = os.path.join(input_folder, raw_data_file_keyword)
        raw_data_files_list = glob.glob(raw_data_glob_pattern.strip(), recursive=True)
        # Searching for matching files with recursion
        if not raw_data_files_list:
            raise ValueError("No files were found in {} that had the keyword {} in it".format(input_folder, raw_data_file_keyword))


In [17]:
raw_data_glob_pattern

'/nancy/projects/dominance_strain_comparison/data/pilot_1/*[rR]eward*'

- Checking to see if only one file is specified

In [18]:
if not raw_data_file_path:
    if len(raw_data_files_list) >= 2:
        raise ValueError("More than one file was found with the path/pattern of {}. Please rerun the previous cell with the correct path".format(raw_data_glob_pattern))
    else:
        # Using the first(and only file) as the file path
        raw_data_file_path = raw_data_files_list[0]

In [19]:
raw_data_file_path

'/nancy/projects/dominance_strain_comparison/data/pilot_1/pilot_1_Reward Competition Video Scoring Assignments.xlsx'

## Getting a list of all the sheets

In [20]:
# Getting the sheet names for the excel file
xls = pd.ExcelFile(raw_data_file_path)
raw_data_sheet_names = xls.sheet_names

In [21]:
raw_data_sheet_names

['Master Master',
 'back up',
 'Sheet2',
 'Master List',
 'Master Day 2',
 'Master Day 1',
 'Meghan',
 'Jocelyn',
 'Erika',
 'Ezra',
 'Naeliz']

# NOTE: If there is a set sheet names used for this notebook, edit the cell below with the name of the sheets each in quotation marks seperated by commas in the brackets
   - i.e. `['CAGE 1', 'CAGE 2', 'CAGE 3', 'CAGE 4']`

In [22]:
inputted_sheet_names_list = []

- Asking the user what sheets they want to use

In [23]:
if not inputted_sheet_names_list:
    inputted_sheet_names_string = input("""Type out the name of the sheets that you want to be processed. 
    Each name must be put in quotes and seperated by a comma(,). i.e. "CAGE3", "CAGE4"

    The available sheets are: {}

    Alternatively, you can use the index of the list of names above. 
    Remember, that Python is zero indexed so the first item will be have the 0 index, second the 1 index, and so on. 
    i.e. 1, 2

    NOTE: If left blank, all sheets will be used
    """.format(raw_data_sheet_names))

Type out the name of the sheets that you want to be processed. 
    Each name must be put in quotes and seperated by a comma(,). i.e. "CAGE3", "CAGE4"

    The available sheets are: ['Master Master', 'back up', 'Sheet2', 'Master List', 'Master Day 2', 'Master Day 1', 'Meghan', 'Jocelyn', 'Erika', 'Ezra', 'Naeliz']

    Alternatively, you can use the index of the list of names above. 
    Remember, that Python is zero indexed so the first item will be have the 0 index, second the 1 index, and so on. 
    i.e. 1, 2

    NOTE: If left blank, all sheets will be used
    'Master Master'


In [24]:
if not inputted_sheet_names_list:
    # Making a list out of the string of inputted sheet names
    if inputted_sheet_names_string:
        inputted_sheet_names_string = "[" + inputted_sheet_names_string + "]"
        # Turning the string into a list
        inputted_sheet_names_list = ast.literal_eval(inputted_sheet_names_string)
    # Using all the sheet names if no sheet name is specified
    else:
        inputted_sheet_names_list =  raw_data_sheet_names

In [25]:
inputted_sheet_names_list

['Master Master']

- Converting all the numbers into the sheet name that the index of the number corresponds to

In [26]:
if inputted_sheet_names_list:
    for index, sheet in enumerate(inputted_sheet_names_list):
        # Checking if the sheet name was a number
        if isinstance(sheet, int) and str(sheet).isdigit():
            inputted_sheet_names_list[index] =  raw_data_sheet_names[sheet]

In [27]:
inputted_sheet_names_list

['Master Master']

- Checking to see if all the sheets are in the excel file

In [28]:
if not set(inputted_sheet_names_list).issubset(raw_data_sheet_names):
    # Getting all the sheets that were not in the original spreadsheet
    not_included_sheet_names = set(inputted_sheet_names_list) - set(raw_data_sheet_names)
    raise ValueError("All the listed sheet names are not in {}".format(not_included_sheet_names))

## Finding the row for the header

- Headers are the row in a spreadsheet that has all the column names. Sometimes spreadsheets don't use the first row as the row with the column names. So, the row for the header will be asked or assumed to be the first row

# NOTE: If there is a set row that has the column names, then enter the row number in the quotation marks

In [29]:
# Making the row number a string, so that "0" is treated as a True value
all_header_row = ""
if not all_header_row:
    all_header_row = False
else:
    all_header_row = int(all_header_row)


In [30]:
sheet_name_to_everything = defaultdict(dict)
for sheet in inputted_sheet_names_list:
    if all_header_row is False:

        print("\nCurrent Sheet Name: {}".format(sheet))    
        per_sheet_dataframe = pd.read_excel(raw_data_file_path, sheet_name=sheet, header=0)

        # Showing the columns that are chosen with the header being the 0th row
        print("Columns Names: {}".format(per_sheet_dataframe.columns))
        # Show the dataframe that would be created with the header being the 0th row
        print("First few rows of this dataframe:")
        print(pd.read_excel(raw_data_file_path, sheet_name=sheet, header=0).head())

        # Allowing the user the choose the row number for the header
        header_row = input("""Type the row number to be used as the header
        (AKA the row with the column name that you want to use.)
        If you want to keep the column names that were displayed, type 0.
        If you want to use a different row, then type the corresponding number. 

        The rows displayed in this cell are dataframes created from Pandas. 
        To use the row with the 0 index for column names, type 1. 
        For the row with the 1 index, it will be 2 and so on. i.e. 2

        If you are looking at the original spread sheet, remember that Python is zero indexed. 
        So the first row will be 0, second will be 1, and so on. 
        i.e. 1

        NOTE: If left blank, the original row that was used will be used.
        """).strip()

        if header_row == "":
            header_row = 0
        else:
            header_row = int(header_row)
    
    # Checking if any of the column names are from empty cells
    column_names = "".join([str(col) for col in pd.read_excel(raw_data_file_path, sheet_name=sheet, header=header_row).columns])
    # If a column name came from an empty cell, it would have "Unnamed" in it
    if "Unnamed" in column_names:
        raise ValueError("""Not all the cells in the chosen row are filled in.
                         Please choose a row that has the name of the columns
                         
                         The values in this row are: {}""".format(column_names))
    else:
        sheet_name_to_everything[sheet]["header_row"] = header_row


Current Sheet Name: Master Master
Columns Names: Index(['Date', 'Cage', 'Box', 'Match', 'Scorer', 'Trial 1 Time',
       'Trial 1 Winner', 'Trial 2 Time ', 'Trial 2 Winner', 'Trial 3 Time',
       'Trial 3 Winner', 'Trial 4 Time', 'Trial 4 Winner', 'Trial 5 Time',
       'Trial 5 Winner', 'Trial 6 Time', 'Trial 6 Winner', 'Trial 7 Time',
       'Trial 7 Winner', 'Trial 8 Time', 'Trial 8 Winner', 'Trial 9 Time',
       'Trial 9 Winner', 'Trial 10 Time ', 'Trial 10 Winner', 'Trial 11 Time',
       'Trial 11 Winner', 'Trial 12 Time ', 'Trial 12 Winner', 'Trial 13 Time',
       'Trial 13 Winner', 'Trial 14 Time', 'Trial 14 Winner', 'Trial 15 Time',
       'Trial 15 Winner', 'Trial 16 Time ', 'Trial 16 Winner', 'Trial 17 Time',
       'Trial 17 Winner', 'Trial 18 Time', 'Trial 18 Winner', 'Trial 19 Time',
       'Trial 19 Winner', 'Trial 20 Time', 'Trial 20 Winner', 'Mouse 1 Wins',
       'Mouse 2 Wins', 'Ties '],
      dtype='object')
First few rows of this dataframe:
        Date  Cage  

In [31]:
sheet_name_to_everything

defaultdict(dict, {'Master Master': {'header_row': 0}})

## Reading in all the spreadsheets

In [32]:
# Going through each sheet and creating a dataframe of it
for key, value in sheet_name_to_everything.items():
    value["original_behavior_recording_dataframe"] = pd.read_excel(raw_data_file_path, sheet_name=key, header=value["header_row"])

In [33]:
value["original_behavior_recording_dataframe"].head()

,Date,Cage,Box,Match,Scorer,Trial 1 Time,Trial 1 Winner,Trial 2 Time,Trial 2 Winner,Trial 3 Time,...,Trial 17 Winner,Trial 18 Time,Trial 18 Winner,Trial 19 Time,Trial 19 Winner,Trial 20 Time,Trial 20 Winner,Mouse 1 Wins,Mouse 2 Wins,Ties
0,2022-05-19,1,1.0,1.1 v 1.2,NaN,1:12-1:22,1.1,2:31-2:41,1.1,4:00-4:10,...,1.1,27:00-27:10,1.2,28:19-28:29,1.1,29:53-30:03,1.1,11.0,0,0.0
1,2022-05-19,1,2.0,1.3 v 1.4,NaN,1:12-1:22,1.4,2:30-2:40,1.4,3:59-4:09,...,1.4,27:01-27:11,1.4,28:20-28:30,1.3,29:53-30:03,1.3,0.0,9,0.0
2,2022-05-19,1,3.0,1.1 v 1.3,NaN,1:07-1:17,1.3,3:56-4:06,tie,5:14-5:24,...,1.3,28:16-28:26,1.3,29:49-29:59,1.3,NaN,NaN,NaN,NaN,NaN
3,2022-05-19,1,4.0,1.2 v 1.4,NaN,1:06-1:16,1.4,2:25-2:35,1.2,3:54-4:04,...,1.2,26:56-27:06,1.4,28:15-28:25,1.4,29:48-29:58,1.4,NaN,checked whole row,NaN
4,2022-05-19,1,2.0,1.2 v 1.3,NaN,1:11-1:21,1.2,2:30-2:40,1.3,4:00-4:10,...,1.3,27:01-27:11,1.2,28:19-28:29,1.2,29:53-30:03,1.2,NaN,checked whole row,NaN


In [34]:
value["original_behavior_recording_dataframe"].tail()

,Date,Cage,Box,Match,Scorer,Trial 1 Time,Trial 1 Winner,Trial 2 Time,Trial 2 Winner,Trial 3 Time,...,Trial 17 Winner,Trial 18 Time,Trial 18 Winner,Trial 19 Time,Trial 19 Winner,Trial 20 Time,Trial 20 Winner,Mouse 1 Wins,Mouse 2 Wins,Ties
43,2022-05-20,4,1.0,4.2 v 4.3,NaN,1:13-1:23,4.3,2:31-2:41,4.3,4:00-4:10,...,4.3,27:01-27:11,4.3,28:20-28:30,4.3,29:54-30:04,4.2,0.0,0,4.0
44,2022-05-20,4,4.0,4.1 v 4.3,NaN,2:56-3:06,4.3,4:15-4:25,4.3,5:44-5:54,...,4.1,28:46-28:56,4.1,30:05-30:15,4.1,NaN,NaN,0.0,0,1.0
45,2022-05-20,4,3.0,4.2 v 4.4,NaN,2:57-3:07,4.4,4:15-4:25,4.4,5:44-5:54,...,4.2,28:46-28:56,4.4,30:05-30:15,4.2,NaN,NaN,NaN,checked whole row,NaN
46,2022-05-20,4,2.0,4.1 v 4.2,NaN,1:11-1:21,4.1,2:30-2:40,4.2,3:58-4:08,...,4.1,27:00-27:10,4.1,28:19-28:29,4.1,29:52-30:02,4.1,NaN,checked whole row,NaN
47,2022-05-20,4,1.0,4.3 v 4.4,NaN,1:12-1:22,4.3,2:30-2:40,4.3,3:58-4:08,...,4.4,27:00-27:10,Tie,28:19-28:29,4.4,29:53-30:03,4.3,0.0,0,1.0


## Standarizing the Columns

- Making all the column names lower case and removing any extra spaces in the beginning and at the end
    - One dictionary per sheet

In [35]:
for key, value in sheet_name_to_everything.items():
    # Creating a dictionary that maps the original column name to the standarized one
    column_name_to_standarized = defaultdict(dict)
    for col in value["original_behavior_recording_dataframe"]:
        # Making the column name lower case and removing the spaces
        column_name_to_standarized[col] = "_".join(str(col).lower().strip().split(" "))
    value["column_name_to_standarized"] = column_name_to_standarized

In [36]:
value["column_name_to_standarized"]

defaultdict(dict,
            {'Date': 'date',
             'Cage': 'cage',
             'Box': 'box',
             'Match': 'match',
             'Scorer': 'scorer',
             'Trial 1 Time': 'trial_1_time',
             'Trial 1 Winner': 'trial_1_winner',
             'Trial 2 Time ': 'trial_2_time',
             'Trial 2 Winner': 'trial_2_winner',
             'Trial 3 Time': 'trial_3_time',
             'Trial 3 Winner': 'trial_3_winner',
             'Trial 4 Time': 'trial_4_time',
             'Trial 4 Winner': 'trial_4_winner',
             'Trial 5 Time': 'trial_5_time',
             'Trial 5 Winner': 'trial_5_winner',
             'Trial 6 Time': 'trial_6_time',
             'Trial 6 Winner': 'trial_6_winner',
             'Trial 7 Time': 'trial_7_time',
             'Trial 7 Winner': 'trial_7_winner',
             'Trial 8 Time': 'trial_8_time',
             'Trial 8 Winner': 'trial_8_winner',
             'Trial 9 Time': 'trial_9_time',
             'Trial 9 Winner': 'tri

In [37]:
# Renaming all the columns to the lower case and space removed version
for key, value in sheet_name_to_everything.items():
    value["processed_behavior_recording_dataframe"] = value["original_behavior_recording_dataframe"].rename(columns=value["column_name_to_standarized"])
    value["processed_behavior_recording_dataframe"]["sheet_name"] = key

In [38]:
value["processed_behavior_recording_dataframe"].head()

,date,cage,box,match,scorer,trial_1_time,trial_1_winner,trial_2_time,trial_2_winner,trial_3_time,...,trial_18_time,trial_18_winner,trial_19_time,trial_19_winner,trial_20_time,trial_20_winner,mouse_1_wins,mouse_2_wins,ties,sheet_name
0,2022-05-19,1,1.0,1.1 v 1.2,NaN,1:12-1:22,1.1,2:31-2:41,1.1,4:00-4:10,...,27:00-27:10,1.2,28:19-28:29,1.1,29:53-30:03,1.1,11.0,0,0.0,Master Master
1,2022-05-19,1,2.0,1.3 v 1.4,NaN,1:12-1:22,1.4,2:30-2:40,1.4,3:59-4:09,...,27:01-27:11,1.4,28:20-28:30,1.3,29:53-30:03,1.3,0.0,9,0.0,Master Master
2,2022-05-19,1,3.0,1.1 v 1.3,NaN,1:07-1:17,1.3,3:56-4:06,tie,5:14-5:24,...,28:16-28:26,1.3,29:49-29:59,1.3,NaN,NaN,NaN,NaN,NaN,Master Master
3,2022-05-19,1,4.0,1.2 v 1.4,NaN,1:06-1:16,1.4,2:25-2:35,1.2,3:54-4:04,...,26:56-27:06,1.4,28:15-28:25,1.4,29:48-29:58,1.4,NaN,checked whole row,NaN,Master Master
4,2022-05-19,1,2.0,1.2 v 1.3,NaN,1:11-1:21,1.2,2:30-2:40,1.3,4:00-4:10,...,27:01-27:11,1.2,28:19-28:29,1.2,29:53-30:03,1.2,NaN,checked whole row,NaN,Master Master


# Add Stuff afterm

In [39]:
reward_competition_df = sheet_name_to_everything[key]["processed_behavior_recording_dataframe"]

In [40]:
reward_competition_df.columns

Index(['date', 'cage', 'box', 'match', 'scorer', 'trial_1_time',
       'trial_1_winner', 'trial_2_time', 'trial_2_winner', 'trial_3_time',
       'trial_3_winner', 'trial_4_time', 'trial_4_winner', 'trial_5_time',
       'trial_5_winner', 'trial_6_time', 'trial_6_winner', 'trial_7_time',
       'trial_7_winner', 'trial_8_time', 'trial_8_winner', 'trial_9_time',
       'trial_9_winner', 'trial_10_time', 'trial_10_winner', 'trial_11_time',
       'trial_11_winner', 'trial_12_time', 'trial_12_winner', 'trial_13_time',
       'trial_13_winner', 'trial_14_time', 'trial_14_winner', 'trial_15_time',
       'trial_15_winner', 'trial_16_time', 'trial_16_winner', 'trial_17_time',
       'trial_17_winner', 'trial_18_time', 'trial_18_winner', 'trial_19_time',
       'trial_19_winner', 'trial_20_time', 'trial_20_winner', 'mouse_1_wins',
       'mouse_2_wins', 'ties', 'sheet_name'],
      dtype='object')

In [41]:
reward_competition_df = reward_competition_df.drop([col for col in reward_competition_df.columns if "wins" in col or "ties" in col], axis=1, errors="ignore")

In [42]:
columns_to_keep = [col for col in reward_competition_df.columns if "time" not in col.lower().strip()]

In [43]:
columns_to_keep

['date',
 'cage',
 'box',
 'match',
 'scorer',
 'trial_1_winner',
 'trial_2_winner',
 'trial_3_winner',
 'trial_4_winner',
 'trial_5_winner',
 'trial_6_winner',
 'trial_7_winner',
 'trial_8_winner',
 'trial_9_winner',
 'trial_10_winner',
 'trial_11_winner',
 'trial_12_winner',
 'trial_13_winner',
 'trial_14_winner',
 'trial_15_winner',
 'trial_16_winner',
 'trial_17_winner',
 'trial_18_winner',
 'trial_19_winner',
 'trial_20_winner',
 'sheet_name']

In [44]:
reward_competition_df = reward_competition_df[columns_to_keep].copy()

In [45]:
reward_competition_df.head()

,date,cage,box,match,scorer,trial_1_winner,trial_2_winner,trial_3_winner,trial_4_winner,trial_5_winner,...,trial_12_winner,trial_13_winner,trial_14_winner,trial_15_winner,trial_16_winner,trial_17_winner,trial_18_winner,trial_19_winner,trial_20_winner,sheet_name
0,2022-05-19,1,1.0,1.1 v 1.2,NaN,1.1,1.1,1.1,1.1,1.2,...,1.2,1.2,1.2,1.2,1.2,1.1,1.2,1.1,1.1,Master Master
1,2022-05-19,1,2.0,1.3 v 1.4,NaN,1.4,1.4,1.3,1.3,1.4,...,1.3,1.3,1.3,1.4,1.4,1.4,1.4,1.3,1.3,Master Master
2,2022-05-19,1,3.0,1.1 v 1.3,NaN,1.3,tie,1.3,1.1,1.1,...,1.3,1.1,1.3,1.3,1.3,1.3,1.3,1.3,NaN,Master Master
3,2022-05-19,1,4.0,1.2 v 1.4,NaN,1.4,1.2,1.2,1.2,1.4,...,1.4,1.2,1.2,1.4,1.2,1.2,1.4,1.4,1.4,Master Master
4,2022-05-19,1,2.0,1.2 v 1.3,NaN,1.2,1.3,1.3,1.3,1.2,...,tie,1.2,1.3,1.2,1.2,1.3,1.2,1.2,1.2,Master Master


In [46]:
reward_competition_df = reward_competition_df.dropna(subset=["trial_1_winner"])

## Add both IDs as a column

In [47]:
reward_competition_df["animal_ids"] = reward_competition_df["match"].apply(lambda x: tuple(sorted([all_ids.strip() for all_ids in re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)])))

In [48]:
reward_competition_df.head()

,date,cage,box,match,scorer,trial_1_winner,trial_2_winner,trial_3_winner,trial_4_winner,trial_5_winner,...,trial_13_winner,trial_14_winner,trial_15_winner,trial_16_winner,trial_17_winner,trial_18_winner,trial_19_winner,trial_20_winner,sheet_name,animal_ids
0,2022-05-19,1,1.0,1.1 v 1.2,NaN,1.1,1.1,1.1,1.1,1.2,...,1.2,1.2,1.2,1.2,1.1,1.2,1.1,1.1,Master Master,"(1.1, 1.2)"
1,2022-05-19,1,2.0,1.3 v 1.4,NaN,1.4,1.4,1.3,1.3,1.4,...,1.3,1.3,1.4,1.4,1.4,1.4,1.3,1.3,Master Master,"(1.3, 1.4)"
2,2022-05-19,1,3.0,1.1 v 1.3,NaN,1.3,tie,1.3,1.1,1.1,...,1.1,1.3,1.3,1.3,1.3,1.3,1.3,NaN,Master Master,"(1.1, 1.3)"
3,2022-05-19,1,4.0,1.2 v 1.4,NaN,1.4,1.2,1.2,1.2,1.4,...,1.2,1.2,1.4,1.2,1.2,1.4,1.4,1.4,Master Master,"(1.2, 1.4)"
4,2022-05-19,1,2.0,1.2 v 1.3,NaN,1.2,1.3,1.3,1.3,1.2,...,1.2,1.3,1.2,1.2,1.3,1.2,1.2,1.2,Master Master,"(1.2, 1.3)"


In [49]:
reward_competition_df["cohort"] = cohort_name

In [50]:
cage_to_strain = {"1": "C57", "2": "C57", "3": "CD1", "4": "CD1"}

# NOTE: If there are strains that are associated to each cage, then create a dictionary of cage numbers to strains inside the `{}`
- i.e. `cage_to_strain = {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6": "CD1"}`

In [51]:
reward_competition_df["strain"] = reward_competition_df["cage"].astype(str).map(cage_to_strain)

In [52]:
all_cages = "_".join([str(cage) for cage in sorted(reward_competition_df["cage"].unique())])

In [53]:
all_cages

'1_2_3_4'

In [54]:
earliest_date = reward_competition_df["date"].min()
latest_date = reward_competition_df["date"].max()

In [55]:
earliest_date = str(earliest_date.date()).replace("-", "")

In [56]:
latest_date = str(latest_date.date()).replace("-", "")

In [57]:
earliest_date

'20220519'

In [58]:
latest_date

'20220520'

- Creating a subfolder to put the Elo rating Spreadsheets

In [59]:
elo_rating_spreadsheet_output_directory = os.path.join(".", "proc", "elo_rating_spread_sheets", "{}".format(protocol_name))

In [60]:
elo_rating_spreadsheet_output_directory

'./proc/elo_rating_spread_sheets/reward_competition'

In [61]:
os.makedirs(elo_rating_spreadsheet_output_directory, exist_ok=True)

In [62]:
file_name = "{}_{}_trial_as_columns_cages_{}_date_{}_{}.csv".format(cohort_name, protocol_name, all_cages, earliest_date, latest_date)

reward_competition_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name))

## Melt 

In [63]:
reward_competition_df["index"] = reward_competition_df.index
reward_competition_df = reward_competition_df.reset_index(drop=True)

In [64]:
reward_competition_df.head()

,date,cage,box,match,scorer,trial_1_winner,trial_2_winner,trial_3_winner,trial_4_winner,trial_5_winner,...,trial_16_winner,trial_17_winner,trial_18_winner,trial_19_winner,trial_20_winner,sheet_name,animal_ids,cohort,strain,index
0,2022-05-19,1,1.0,1.1 v 1.2,NaN,1.1,1.1,1.1,1.1,1.2,...,1.2,1.1,1.2,1.1,1.1,Master Master,"(1.1, 1.2)",pilot_1,C57,0
1,2022-05-19,1,2.0,1.3 v 1.4,NaN,1.4,1.4,1.3,1.3,1.4,...,1.4,1.4,1.4,1.3,1.3,Master Master,"(1.3, 1.4)",pilot_1,C57,1
2,2022-05-19,1,3.0,1.1 v 1.3,NaN,1.3,tie,1.3,1.1,1.1,...,1.3,1.3,1.3,1.3,NaN,Master Master,"(1.1, 1.3)",pilot_1,C57,2
3,2022-05-19,1,4.0,1.2 v 1.4,NaN,1.4,1.2,1.2,1.2,1.4,...,1.2,1.2,1.4,1.4,1.4,Master Master,"(1.2, 1.4)",pilot_1,C57,3
4,2022-05-19,1,2.0,1.2 v 1.3,NaN,1.2,1.3,1.3,1.3,1.2,...,1.2,1.3,1.2,1.2,1.2,Master Master,"(1.2, 1.3)",pilot_1,C57,4


In [65]:
melted_reward_competition_df = reward_competition_df.melt(id_vars=["index", "date", "cage", "box", "match", "scorer", "animal_ids"], 
        var_name="trial", 
        value_name="winner")

In [66]:
melted_reward_competition_df.head()

,index,date,cage,box,match,scorer,animal_ids,trial,winner
0,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_1_winner,1.1
1,1,2022-05-19,1,2.0,1.3 v 1.4,NaN,"(1.3, 1.4)",trial_1_winner,1.4
2,2,2022-05-19,1,3.0,1.1 v 1.3,NaN,"(1.1, 1.3)",trial_1_winner,1.3
3,3,2022-05-19,1,4.0,1.2 v 1.4,NaN,"(1.2, 1.4)",trial_1_winner,1.4
4,4,2022-05-19,1,2.0,1.2 v 1.3,NaN,"(1.2, 1.3)",trial_1_winner,1.2


- Dropping all rows that don't contain Winner

In [67]:
melted_reward_competition_df = melted_reward_competition_df.dropna(subset="winner")

- Making all the ids into string

In [68]:
melted_reward_competition_df["winner"] = melted_reward_competition_df["winner"].astype(str)
melted_reward_competition_df["winner"] = melted_reward_competition_df["winner"].apply(lambda x: x.lower().strip())

In [69]:
melted_reward_competition_df["match"].unique()

array(['1.1 v 1.2', '1.3 v 1.4', '1.1 v 1.3', '1.2 v 1.4', '1.2 v 1.3',
       '1.1 v 1.4', '2.1 v 2.2', '2.3 v 2.4', '2.1 v 2.3', '2.2 v 2.4',
       '2.1 v 2.4', '2.2 v 2.3', '3.1 v 3.2', '3.3 v 3.4', '3.1 v 3.3',
       '3.2 v 3.4', '3.1 v 3.4', '3.2 v 3.3', '4.1 v 4.2', '4.3 v 4.4',
       '4.1 v 4.3', '4.2 v 4.4', '4.1 v 4.4', '4.2 v 4.3'], dtype=object)

- Making a different column for ties

In [70]:
melted_reward_competition_df["tie"] = melted_reward_competition_df["winner"].apply(lambda x: True if "tie" in x.lower().strip() else False)

- Replacing tie with the first animal id for elo score plotting

In [71]:
melted_reward_competition_df["winner"] = melted_reward_competition_df.apply(lambda x: x["animal_ids"][0] if x["tie"] else x["winner"], axis=1)

In [72]:
melted_reward_competition_df[melted_reward_competition_df["tie"]]

,index,date,cage,box,match,scorer,animal_ids,trial,winner,tie
29,29,2022-05-19,3,2.0,3.2 v 3.3,NaN,"(3.2, 3.3)",trial_1_winner,3.2,True
32,32,2022-05-20,3,2.0,3.1 v 3.3,NaN,"(3.1, 3.3)",trial_1_winner,3.1,True
50,2,2022-05-19,1,3.0,1.1 v 1.3,NaN,"(1.1, 1.3)",trial_2_winner,1.1,True
56,8,2022-05-20,1,2.0,1.1 v 1.3,NaN,"(1.1, 1.3)",trial_2_winner,1.1,True
58,10,2022-05-20,1,4.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_2_winner,1.1,True
...,...,...,...,...,...,...,...,...,...,...
863,47,2022-05-20,4,1.0,4.3 v 4.4,NaN,"(4.3, 4.4)",trial_18_winner,4.3,True
874,10,2022-05-20,1,4.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_19_winner,1.1,True
884,20,2022-05-20,2,4.0,2.1 v 2.3,NaN,"(2.1, 2.3)",trial_19_winner,2.1,True
885,21,2022-05-20,2,3.0,2.2 v 2.4,NaN,"(2.2, 2.4)",trial_19_winner,2.2,True


- Dropping all non tie and competition rows

In [73]:
melted_reward_competition_df["winner_is_number"] = melted_reward_competition_df["winner"].apply(lambda x: True if re.match(r'^-?\d+(?:\.\d+)$', x) else False)


In [74]:
melted_reward_competition_df = melted_reward_competition_df[melted_reward_competition_df["winner_is_number"] == True]
melted_reward_competition_df = melted_reward_competition_df.drop(columns=["winner_is_number"])

In [75]:
melted_reward_competition_df.head()

,index,date,cage,box,match,scorer,animal_ids,trial,winner,tie
0,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_1_winner,1.1,False
1,1,2022-05-19,1,2.0,1.3 v 1.4,NaN,"(1.3, 1.4)",trial_1_winner,1.4,False
2,2,2022-05-19,1,3.0,1.1 v 1.3,NaN,"(1.1, 1.3)",trial_1_winner,1.3,False
3,3,2022-05-19,1,4.0,1.2 v 1.4,NaN,"(1.2, 1.4)",trial_1_winner,1.4,False
4,4,2022-05-19,1,2.0,1.2 v 1.3,NaN,"(1.2, 1.3)",trial_1_winner,1.2,False


- Getting the number of the trial so that we can order by number(instead of string, which would make 11 come before 2)

In [76]:
melted_reward_competition_df["trial_number"] = melted_reward_competition_df["trial"].apply(lambda x: int(x.lower().strip("trial").strip("winner").strip("_")))

In [77]:
melted_reward_competition_df = melted_reward_competition_df.sort_values(["index", "trial_number"])

In [78]:
melted_reward_competition_df.head()

,index,date,cage,box,match,scorer,animal_ids,trial,winner,tie,trial_number
0,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_1_winner,1.1,False,1
48,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_2_winner,1.1,False,2
96,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_3_winner,1.1,False,3
144,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_4_winner,1.1,False,4
192,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_5_winner,1.2,False,5


In [79]:
melted_reward_competition_df["trial_number"].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

## Add a column for the loser ID

In [80]:
melted_reward_competition_df.columns

Index(['index', 'date', 'cage', 'box', 'match', 'scorer', 'animal_ids',
       'trial', 'winner', 'tie', 'trial_number'],
      dtype='object')

In [81]:
melted_reward_competition_df.head()

,index,date,cage,box,match,scorer,animal_ids,trial,winner,tie,trial_number
0,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_1_winner,1.1,False,1
48,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_2_winner,1.1,False,2
96,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_3_winner,1.1,False,3
144,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_4_winner,1.1,False,4
192,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_5_winner,1.2,False,5


In [82]:
melted_reward_competition_df["loser"] = melted_reward_competition_df.apply(lambda x: (list(set(x["animal_ids"]) - set([x["winner"]]))[0]), axis=1)

In [83]:
melted_reward_competition_df["loser"].unique()

array(['1.2', '1.1', '1.3', '1.4', '2.2', '2.1', '2.4', '2.3', '3.2',
       '3.1', '3.4', '3.3', '4.2', '4.1', '4.4', '4.3'], dtype=object)

In [84]:
melted_reward_competition_df.head(n=5)

,index,date,cage,box,match,scorer,animal_ids,trial,winner,tie,trial_number,loser
0,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_1_winner,1.1,False,1,1.2
48,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_2_winner,1.1,False,2,1.2
96,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_3_winner,1.1,False,3,1.2
144,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_4_winner,1.1,False,4,1.2
192,0,2022-05-19,1,1.0,1.1 v 1.2,NaN,"(1.1, 1.2)",trial_5_winner,1.2,False,5,1.1


In [85]:
melted_reward_competition_df.tail(n=5)

,index,date,cage,box,match,scorer,animal_ids,trial,winner,tie,trial_number,loser
767,47,2022-05-20,4,1.0,4.3 v 4.4,NaN,"(4.3, 4.4)",trial_16_winner,4.3,False,16,4.4
815,47,2022-05-20,4,1.0,4.3 v 4.4,NaN,"(4.3, 4.4)",trial_17_winner,4.4,False,17,4.3
863,47,2022-05-20,4,1.0,4.3 v 4.4,NaN,"(4.3, 4.4)",trial_18_winner,4.3,True,18,4.4
911,47,2022-05-20,4,1.0,4.3 v 4.4,NaN,"(4.3, 4.4)",trial_19_winner,4.4,False,19,4.3
959,47,2022-05-20,4,1.0,4.3 v 4.4,NaN,"(4.3, 4.4)",trial_20_winner,4.3,False,20,4.4


# ADD stuff before this

# Finding the rows with ties

- If a trial is a tie, there should be another column that indicates that it is a tie. The points will be counted for less. If there is no column, then none of the trials will be counted as ties.

# NOTE: If there is a set column that denotes whether the match has a winner or not, then replace the `None` with the name of the column with quotation marks

In [86]:
all_ties_column = None

In [87]:
for key, value in sheet_name_to_everything.items():
    
    if all_ties_column is None:
        value["ties_column"] = None
    elif all_ties_column:
        value["ties_column"] = all_ties_column
    else:
    
        # Asking users to specify which column is the one with the winner's information
        value["ties_column"] = input("""Type the name of the column that specifies whether or not a tie has occured
        i.e. "tie"

        The available columns are: {}

        Note: If left blank, the column with "ties" in the name will be used as the column. 
        If there are none, then this part will be skipped

        """.format(value["processed_behavior_recording_dataframe"].columns))

        # Looks for columns with "tie" in them if none of inputted
        if not value["ties_column"]:
            if [col for col in value["processed_behavior_recording_dataframe"].columns if "tie" in col.lower()]:
                value["ties_column"] = [col for col in value["processed_behavior_recording_dataframe"].columns if "tie" in col.lower()][0]
            else: 
                value["ties_column"] = None
    current_processed_behavior_recording_dataframe = value["processed_behavior_recording_dataframe"].copy()
    try:
        # Standarizing the name of the tie column
        value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].rename(columns={value["ties_column"]: "match_is_tie"})
        value["ties_column"] = "match_is_tie"
        # Fillling in the tie column with 0s so that we can fill in the other columns with the previous values
        value["processed_behavior_recording_dataframe"][value["ties_column"]] = value["processed_behavior_recording_dataframe"][value["ties_column"]].fillna()
        # Converting all the tie values into bool so we can filter for the cells with ties
        value["processed_behavior_recording_dataframe"][value["ties_column"]] = value["processed_behavior_recording_dataframe"][value["ties_column"]].astype(bool)
    except:
        value["processed_behavior_recording_dataframe"] = current_processed_behavior_recording_dataframe
        value["ties_column"] = None

## Adding the session number

- We are adding the session number to all the trials. The session number is counting the number of recording sessions that have happened up until that trial. Usually, each session in the spreadsheet is divided up by a session's first row having the date filled in. So we will label a new session when a date is filled in.

# NOTE: If there is a set column that divides the rows up by session, then enter it in the cell below between the quotation marks. Default is `"date"`

# TODO: Recheck starting here

In [88]:
all_session_divider_column = ""

In [89]:
for key, value in sheet_name_to_everything.items():
    if not all_session_divider_column:
        # Allowing the user to choose the column that indicates a new session
        session_divider_column = input("""Type the name of the column to be used to divide the rows into sessions. 
        Pick a column that has a value in the cell only with the first row of each session.
        This is usually the "date" column.

        If left blank, the default session divider column will be used. The default is "date"

        The available columns are: {}

        NOTE: If left blank, the column with "date" in the name will be used as the column
        """.format(value["processed_behavior_recording_dataframe"].columns))
    else:
        session_divider_column = all_session_divider_column
        
    # Making the column name lowercase and removing any punctuation
    value["session_divider_column"] = session_divider_column.lower().strip('"').strip("'").strip()

    # Looks for columns with "date" in them if none is inputted
    if not value["session_divider_column"]:
        value["session_divider_column"] = [col for col in value["processed_behavior_recording_dataframe"].columns if "date" in col.lower()][0]

    # Checks if that column is in the dataframe
    if value["session_divider_column"] not in value["processed_behavior_recording_dataframe"].columns:
        print("WARNING: {} is not a column in {}".format(value["session_divider_column"], key))
        warnings.warn("Look at warning from above or below")
        value["session_divider_column"] = None
    
    # Standarizing all the session divider columns so that they are named date
    if value["session_divider_column"]:
        value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].rename(columns={value["session_divider_column"]: "date"})
        value["session_divider_column"] = "date"

Type the name of the column to be used to divide the rows into sessions. 
        Pick a column that has a value in the cell only with the first row of each session.
        This is usually the "date" column.

        If left blank, the default session divider column will be used. The default is "date"

        The available columns are: Index(['date', 'cage', 'box', 'match', 'scorer', 'trial_1_time',
       'trial_1_winner', 'trial_2_time', 'trial_2_winner', 'trial_3_time',
       'trial_3_winner', 'trial_4_time', 'trial_4_winner', 'trial_5_time',
       'trial_5_winner', 'trial_6_time', 'trial_6_winner', 'trial_7_time',
       'trial_7_winner', 'trial_8_time', 'trial_8_winner', 'trial_9_time',
       'trial_9_winner', 'trial_10_time', 'trial_10_winner', 'trial_11_time',
       'trial_11_winner', 'trial_12_time', 'trial_12_winner', 'trial_13_time',
       'trial_13_winner', 'trial_14_time', 'trial_14_winner', 'trial_15_time',
       'trial_15_winner', 'trial_16_time', 'trial_16_winner

# TODO: Make this standarized between sheets and get difference for each row

In [90]:
for key, value in sheet_name_to_everything.items():
    current_processed_behavior_recording_dataframe = value["processed_behavior_recording_dataframe"].copy()
    try:
        value["processed_behavior_recording_dataframe"][value["session_divider_column"]] = value["processed_behavior_recording_dataframe"][value["session_divider_column"]].fillna(method='ffill')

    except:
        print("WARNING: {} does not have a session divider column".format(key))
        warnings.warn("Look at warning from above or below")
        value["processed_behavior_recording_dataframe"] = current_processed_behavior_recording_dataframe

In [91]:
value["processed_behavior_recording_dataframe"].head()

,date,cage,box,match,scorer,trial_1_time,trial_1_winner,trial_2_time,trial_2_winner,trial_3_time,...,trial_18_time,trial_18_winner,trial_19_time,trial_19_winner,trial_20_time,trial_20_winner,mouse_1_wins,mouse_2_wins,ties,sheet_name
0,2022-05-19,1,1.0,1.1 v 1.2,NaN,1:12-1:22,1.1,2:31-2:41,1.1,4:00-4:10,...,27:00-27:10,1.2,28:19-28:29,1.1,29:53-30:03,1.1,11.0,0,0.0,Master Master
1,2022-05-19,1,2.0,1.3 v 1.4,NaN,1:12-1:22,1.4,2:30-2:40,1.4,3:59-4:09,...,27:01-27:11,1.4,28:20-28:30,1.3,29:53-30:03,1.3,0.0,9,0.0,Master Master
2,2022-05-19,1,3.0,1.1 v 1.3,NaN,1:07-1:17,1.3,3:56-4:06,tie,5:14-5:24,...,28:16-28:26,1.3,29:49-29:59,1.3,NaN,NaN,NaN,NaN,NaN,Master Master
3,2022-05-19,1,4.0,1.2 v 1.4,NaN,1:06-1:16,1.4,2:25-2:35,1.2,3:54-4:04,...,26:56-27:06,1.4,28:15-28:25,1.4,29:48-29:58,1.4,NaN,checked whole row,NaN,Master Master
4,2022-05-19,1,2.0,1.2 v 1.3,NaN,1:11-1:21,1.2,2:30-2:40,1.3,4:00-4:10,...,27:01-27:11,1.2,28:19-28:29,1.2,29:53-30:03,1.2,NaN,checked whole row,NaN,Master Master


In [92]:
value["processed_behavior_recording_dataframe"].tail()

,date,cage,box,match,scorer,trial_1_time,trial_1_winner,trial_2_time,trial_2_winner,trial_3_time,...,trial_18_time,trial_18_winner,trial_19_time,trial_19_winner,trial_20_time,trial_20_winner,mouse_1_wins,mouse_2_wins,ties,sheet_name
43,2022-05-20,4,1.0,4.2 v 4.3,NaN,1:13-1:23,4.3,2:31-2:41,4.3,4:00-4:10,...,27:01-27:11,4.3,28:20-28:30,4.3,29:54-30:04,4.2,0.0,0,4.0,Master Master
44,2022-05-20,4,4.0,4.1 v 4.3,NaN,2:56-3:06,4.3,4:15-4:25,4.3,5:44-5:54,...,28:46-28:56,4.1,30:05-30:15,4.1,NaN,NaN,0.0,0,1.0,Master Master
45,2022-05-20,4,3.0,4.2 v 4.4,NaN,2:57-3:07,4.4,4:15-4:25,4.4,5:44-5:54,...,28:46-28:56,4.4,30:05-30:15,4.2,NaN,NaN,NaN,checked whole row,NaN,Master Master
46,2022-05-20,4,2.0,4.1 v 4.2,NaN,1:11-1:21,4.1,2:30-2:40,4.2,3:58-4:08,...,27:00-27:10,4.1,28:19-28:29,4.1,29:52-30:02,4.1,NaN,checked whole row,NaN,Master Master
47,2022-05-20,4,1.0,4.3 v 4.4,NaN,1:12-1:22,4.3,2:30-2:40,4.3,3:58-4:08,...,27:00-27:10,Tie,28:19-28:29,4.4,29:53-30:03,4.3,0.0,0,1.0,Master Master


# Getting the Session number differences

- Getting the indexes of where each new session starts. So that we can add the session number to each row.

# TODO: Can remove set about not including tie columns

In [93]:
# Example of the columns that will be kept, removing the ties column
list(set(value["processed_behavior_recording_dataframe"].columns) - set([value["ties_column"]]))

['trial_7_winner',
 'trial_2_winner',
 'trial_17_winner',
 'date',
 'trial_7_time',
 'trial_13_time',
 'trial_14_winner',
 'trial_10_time',
 'trial_2_time',
 'trial_8_winner',
 'trial_3_time',
 'match',
 'trial_15_time',
 'trial_5_winner',
 'trial_1_winner',
 'trial_12_winner',
 'trial_18_time',
 'sheet_name',
 'trial_19_winner',
 'trial_20_time',
 'box',
 'trial_9_time',
 'trial_18_winner',
 'trial_1_time',
 'trial_19_time',
 'trial_12_time',
 'trial_11_winner',
 'trial_14_time',
 'trial_8_time',
 'trial_6_time',
 'mouse_1_wins',
 'trial_3_winner',
 'ties',
 'trial_6_winner',
 'trial_20_winner',
 'mouse_2_wins',
 'trial_16_winner',
 'trial_5_time',
 'trial_9_winner',
 'trial_13_winner',
 'trial_15_winner',
 'scorer',
 'trial_17_time',
 'trial_4_time',
 'trial_11_time',
 'trial_4_winner',
 'trial_10_winner',
 'trial_16_time',
 'cage']

In [94]:
for key, value in sheet_name_to_everything.items():
    # Removing tie columns because not all rows are ties, so we do not want to fill them in
    if value["ties_column"]:
        non_ties_columns = list(set(value["processed_behavior_recording_dataframe"].columns) - set([value["ties_column"]]))
        value["processed_behavior_recording_dataframe"][non_ties_columns] = value["processed_behavior_recording_dataframe"][non_ties_columns].fillna(method='ffill')
    else:
        # Filling all the empty cells with the value in the previous cell
        value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].fillna(method='ffill')

    try:
        # Seeing which rows have a different session than the previous one
        # This will be used to plot vertical lines for each new session
        value["processed_behavior_recording_dataframe"]["session_number_difference"] = value["processed_behavior_recording_dataframe"][value["session_divider_column"]].astype('category').cat.codes.diff()

    
    except:
        print("WARNING: {} does not have a session divider column".format(key))
        warnings.warn("Look at warning from above or below")

In [95]:
value["processed_behavior_recording_dataframe"].head(n=15)

,date,cage,box,match,scorer,trial_1_time,trial_1_winner,trial_2_time,trial_2_winner,trial_3_time,...,trial_18_winner,trial_19_time,trial_19_winner,trial_20_time,trial_20_winner,mouse_1_wins,mouse_2_wins,ties,sheet_name,session_number_difference
0,2022-05-19,1,1.0,1.1 v 1.2,NaN,1:12-1:22,1.1,2:31-2:41,1.1,4:00-4:10,...,1.2,28:19-28:29,1.1,29:53-30:03,1.1,11.0,0,0.0,Master Master,NaN
1,2022-05-19,1,2.0,1.3 v 1.4,NaN,1:12-1:22,1.4,2:30-2:40,1.4,3:59-4:09,...,1.4,28:20-28:30,1.3,29:53-30:03,1.3,0.0,9,0.0,Master Master,0.0
2,2022-05-19,1,3.0,1.1 v 1.3,NaN,1:07-1:17,1.3,3:56-4:06,tie,5:14-5:24,...,1.3,29:49-29:59,1.3,29:53-30:03,1.3,0.0,9,0.0,Master Master,0.0
3,2022-05-19,1,4.0,1.2 v 1.4,NaN,1:06-1:16,1.4,2:25-2:35,1.2,3:54-4:04,...,1.4,28:15-28:25,1.4,29:48-29:58,1.4,0.0,checked whole row,0.0,Master Master,0.0
4,2022-05-19,1,2.0,1.2 v 1.3,NaN,1:11-1:21,1.2,2:30-2:40,1.3,4:00-4:10,...,1.2,28:19-28:29,1.2,29:53-30:03,1.2,0.0,checked whole row,0.0,Master Master,0.0
5,2022-05-19,1,1.0,1.1 v 1.4,NaN,1.12-1.22,1.4,1.30-1.40,1.1,4.00-4.10,...,1.4,28.20-28.30,1.4,29.53-30.03,1.1,0.0,checked whole row,0.0,Master Master,0.0
6,2022-05-20,1,4.0,1.1 v 1.4,NaN,1:08-1:18,1.1,2:28-2:38,1.1,3:56-4:06,...,Tie,28:17-28:27,1.4,NO TRIAL,NO TRIAL,16.0,2,1.0,Master Master,1.0
7,2022-05-20,1,3.0,1.2 v 1.3,NaN,1:10-1:20,1.2,2:28-2:38,1.2,3:57-4:07,...,1.2,28:19-28:29,1.3,NO TRIAL,NO TRIAL,16.0,2,1.0,Master Master,0.0
8,2022-05-20,1,2.0,1.1 v 1.3,NaN,1:05-1:15,1.1,2:24-2:34,tie,3:53-4:03,...,1.1,28:13-28:23,1.1,29:47-29:57,1.1,16.0,checked whole row,1.0,Master Master,0.0
9,2022-05-20,1,1.0,1.2 v 1.4,NaN,1:05-1:15,1.2,2:26-2:36,1.4,3:52-4:02,...,1.2,28:15-28_26,1.2,29:47-29:57,1.1,16.0,checked whole row,1.0,Master Master,0.0


In [96]:
value["processed_behavior_recording_dataframe"].head(n=15)

,date,cage,box,match,scorer,trial_1_time,trial_1_winner,trial_2_time,trial_2_winner,trial_3_time,...,trial_18_winner,trial_19_time,trial_19_winner,trial_20_time,trial_20_winner,mouse_1_wins,mouse_2_wins,ties,sheet_name,session_number_difference
0,2022-05-19,1,1.0,1.1 v 1.2,NaN,1:12-1:22,1.1,2:31-2:41,1.1,4:00-4:10,...,1.2,28:19-28:29,1.1,29:53-30:03,1.1,11.0,0,0.0,Master Master,NaN
1,2022-05-19,1,2.0,1.3 v 1.4,NaN,1:12-1:22,1.4,2:30-2:40,1.4,3:59-4:09,...,1.4,28:20-28:30,1.3,29:53-30:03,1.3,0.0,9,0.0,Master Master,0.0
2,2022-05-19,1,3.0,1.1 v 1.3,NaN,1:07-1:17,1.3,3:56-4:06,tie,5:14-5:24,...,1.3,29:49-29:59,1.3,29:53-30:03,1.3,0.0,9,0.0,Master Master,0.0
3,2022-05-19,1,4.0,1.2 v 1.4,NaN,1:06-1:16,1.4,2:25-2:35,1.2,3:54-4:04,...,1.4,28:15-28:25,1.4,29:48-29:58,1.4,0.0,checked whole row,0.0,Master Master,0.0
4,2022-05-19,1,2.0,1.2 v 1.3,NaN,1:11-1:21,1.2,2:30-2:40,1.3,4:00-4:10,...,1.2,28:19-28:29,1.2,29:53-30:03,1.2,0.0,checked whole row,0.0,Master Master,0.0
5,2022-05-19,1,1.0,1.1 v 1.4,NaN,1.12-1.22,1.4,1.30-1.40,1.1,4.00-4.10,...,1.4,28.20-28.30,1.4,29.53-30.03,1.1,0.0,checked whole row,0.0,Master Master,0.0
6,2022-05-20,1,4.0,1.1 v 1.4,NaN,1:08-1:18,1.1,2:28-2:38,1.1,3:56-4:06,...,Tie,28:17-28:27,1.4,NO TRIAL,NO TRIAL,16.0,2,1.0,Master Master,1.0
7,2022-05-20,1,3.0,1.2 v 1.3,NaN,1:10-1:20,1.2,2:28-2:38,1.2,3:57-4:07,...,1.2,28:19-28:29,1.3,NO TRIAL,NO TRIAL,16.0,2,1.0,Master Master,0.0
8,2022-05-20,1,2.0,1.1 v 1.3,NaN,1:05-1:15,1.1,2:24-2:34,tie,3:53-4:03,...,1.1,28:13-28:23,1.1,29:47-29:57,1.1,16.0,checked whole row,1.0,Master Master,0.0
9,2022-05-20,1,1.0,1.2 v 1.4,NaN,1:05-1:15,1.2,2:26-2:36,1.4,3:52-4:02,...,1.2,28:15-28_26,1.2,29:47-29:57,1.1,16.0,checked whole row,1.0,Master Master,0.0


In [97]:
value["processed_behavior_recording_dataframe"].tail(n=15)

,date,cage,box,match,scorer,trial_1_time,trial_1_winner,trial_2_time,trial_2_winner,trial_3_time,...,trial_18_winner,trial_19_time,trial_19_winner,trial_20_time,trial_20_winner,mouse_1_wins,mouse_2_wins,ties,sheet_name,session_number_difference
33,2022-05-20,3,1.0,3.2 v 3.4,NaN,1:08-1:18,3.2,2:27-2:37,3.2,3:55-4:05,...,3.4,28:16-28:26,3.4,NO TRIAL,NO TRIAL,0.0,0,1.0,Master Master,0.0
34,2022-05-20,3,3.0,3.3 v 3.4,NaN,1:31-1:41,3.4,2:50-3:00,3.3,4:18-4:28,...,3.4,28:39-28:49,3.4,30:14-30:24,3.4,0.0,checked whole row,1.0,Master Master,0.0
35,2022-05-20,3,1.4,3.1 v 3.4,NaN,1:14-1:25,3.4,2:33-2:43,3.4,4:02-4:13,...,3.1,28:22-28:32,3.1,29:56-30:06,3.1,0.0,0,0.0,Master Master,0.0
36,2022-05-19,4,3.0,4.1 v 4.2,NaN,1:14-1:24,4.1,2:33-2:43,4.1,4:02-4:12,...,4.1,28:23-28:33,4.1,29:57-30:07,4.1,0.0,0,0.0,Master Master,-1.0
37,2022-05-19,4,4.0,4.3 v 4.4,NaN,1.14-1.24,4.3,1.33-1.43,4.4,4.03-4.13,...,4.3,28.23-28.33,4.3,29.57-30.07,4.3,0.0,0,0.0,Master Master,0.0
38,2022-05-19,4,1.0,4.1 v 4.3,NaN,1:26-1:36,4.1,2:45-2:55,4.1,4:14-4:24,...,4.1,28:34-28:44,4.3,29.57-30.07,4.3,0.0,0,3.0,Master Master,0.0
39,2022-05-19,4,2.0,4.2 v 4.4,NaN,1:27-1:37,4.4,2:45-2:55,4.2,4:14-4:24,...,4.4,28:34-28:44,4.2,NO TRIAL,NO TRIAL,0.0,0,0.0,Master Master,0.0
40,2022-05-19,4,3.0,4.1 v 4.4,NaN,1:12-1:22,4.1,2:31-2:41,4.1,4:00-4:10,...,4.1,28:20-28:30,4.4,29:54-30:04,4.4,0.0,checked whole row,0.0,Master Master,0.0
41,2022-05-19,4,4.0,4.2 v 4.3,NaN,2:22-2:32,4.2,3:41-3:51,4.2,5:09-5:19,...,4.3,29:30-29:40,4.2,29:54-30:04,4.4,0.0,0,2.0,Master Master,0.0
42,2022-05-20,4,2.0,4.1 v 4.4,NaN,1.13-1.23,4.1,1.32-1.42,4.1,4.01-4.11,...,4.1,28.21-28.31,4.4,29.55-30.05,4.4,0.0,0,2.0,Master Master,1.0


## Getting the cage number

- The cage number is sometimes not consistent throughout a spreadsheet for the same cage. So we will try to standarize it into one value.
    - **NOTE**: This assumes cage numbers are actual numbers. And not entirely consisting of letters. If that isn't the case, then you must edit this cell for your needs.

# NOTE: If there is a column that has the cage number information, replace the `None` with the column name in quotation marks

# TODO: Refactor starting here

- Change this when you have winner and loser cage

In [98]:
cage_num_of_winner_column = None
cage_num_of_loser_column = None

- Change this when you have id to cage

In [99]:
id_to_cage = {}

In [100]:
for key, value in sheet_name_to_everything.items():   
    # When you have a dictionary of IDs to Cage Numbers
    if id_to_cage:
        # Specifying the name by default
        value["cage_num_of_winner_column"] = "original_cage_num_of_winner"
        value["cage_num_of_loser_column"] = "original_cage_num_of_loser"
        # Mapping the IDs to the cage number
        value["processed_behavior_recording_dataframe"][value["cage_num_of_winner_column"]] = value["processed_behavior_recording_dataframe"][value["winner_id_column"]].map(id_to_cage)
        value["processed_behavior_recording_dataframe"][value["cage_num_of_loser_column"]] = value["processed_behavior_recording_dataframe"][value["loser_id_column"]].map(id_to_cage)
        continue
        
    # When you have a column that specifies the cage number
    elif cage_num_of_winner_column and cage_num_of_loser_column:
        value["cage_num_of_winner_column"] = cage_num_of_winner_column
        value["cage_num_of_loser_column"] = cage_num_of_loser_column
    elif cage_num_of_winner_column:
        value["cage_num_of_winner_column"] = cage_num_of_winner_column
    elif cage_num_of_loser_column:
        value["cage_num_of_loser_column"] = cage_num_of_loser_column

    
    if not cage_num_of_winner_column:
        # Asking the user which column has the cage number
        value["cage_num_of_winner_column"] = input("""Type the name of the column of the cage of the WINNER subject
        i.e. "cage_num_of_winner"

        The available columns are: {}

        Note: If left blank, the column with "winner" and "cage" will be used as the column. If there is none, then the sheet name will be used.
        """.format(value["processed_behavior_recording_dataframe"].columns))        
        
        # Getting the column that has "winner" and "cage" in its name if no column is specified
        if not value["cage_num_of_winner_column"]:
            # Checking to see if there are any columns with the winner and cage in the name
            column_names_with_cage = [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower() and "winner" in col.lower()]
            # Using the column with cage in the name if there are none with winner and cage
            column_names_with_cage += [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower()]
            if column_names_with_cage:
                value["winnner_cage_column"] = column_names_with_cage[0]
            else:
                # Using the default name as the cage column name if there are none
                value["cage_num_of_winner_column"] = "original_cage_num_of_winner"
    
    if not cage_num_of_loser_column:
        # Asking the user which column has the cage number
        value["cage_num_of_loser_column"] = input("""Type the name of the column of the cage of the LOSER subject
        i.e. "cage_num_of_loser"

        The available columns are: {}

        Note: If left blank, the column with "loser" and "cage" will be used as the column. If there is none, then the sheet name will be used.
        """.format(value["processed_behavior_recording_dataframe"].columns))        
        
        # Getting the column that has "loser" and "cage" in its name if no column is specified
        if not value["cage_num_of_loser_column"]:
            # Checking to see if there are any columns with the loser and cage in the name
            column_names_with_cage = [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower() and "loser" in col.lower()]
            # Using the column with cage in the name if there are none with loser and cage
            column_names_with_cage += [col for col in value["processed_behavior_recording_dataframe"].columns if "cage" in col.lower()]
            if column_names_with_cage:
                value["winnner_cage_column"] = column_names_with_cage[0]
            else:
                # Using the default name as the cage column name if there are none
                value["cage_num_of_loser_column"] = "original_cage_num_of_loser"
    
    # Using the spreadsheet name as the cage number if there are no columns that match
    if value["cage_num_of_winner_column"] not in value["processed_behavior_recording_dataframe"].columns:
        value["processed_behavior_recording_dataframe"][value["cage_num_of_winner_column"]] = key

    if value["cage_num_of_loser_column"] not in value["processed_behavior_recording_dataframe"].columns:
        value["processed_behavior_recording_dataframe"][value["cage_num_of_loser_column"]] = key       
    
    # Turning the cage names into the float number only
    # Or using the same name if there are no floats
    value["processed_behavior_recording_dataframe"]["processed_cage_num_of_winner"] = value["processed_behavior_recording_dataframe"][value["cage_num_of_winner_column"]].apply(lambda x: re.findall(r'\d+', str(x))[0] if re.findall(r'\d+', str(x)) else x)
    value["processed_behavior_recording_dataframe"]["processed_cage_num_of_loser"] = value["processed_behavior_recording_dataframe"][value["cage_num_of_loser_column"]].apply(lambda x: re.findall(r'\d+', str(x))[0] if re.findall(r'\d+', str(x)) else x)

    

Type the name of the column of the cage of the WINNER subject
        i.e. "cage_num_of_winner"

        The available columns are: Index(['date', 'cage', 'box', 'match', 'scorer', 'trial_1_time',
       'trial_1_winner', 'trial_2_time', 'trial_2_winner', 'trial_3_time',
       'trial_3_winner', 'trial_4_time', 'trial_4_winner', 'trial_5_time',
       'trial_5_winner', 'trial_6_time', 'trial_6_winner', 'trial_7_time',
       'trial_7_winner', 'trial_8_time', 'trial_8_winner', 'trial_9_time',
       'trial_9_winner', 'trial_10_time', 'trial_10_winner', 'trial_11_time',
       'trial_11_winner', 'trial_12_time', 'trial_12_winner', 'trial_13_time',
       'trial_13_winner', 'trial_14_time', 'trial_14_winner', 'trial_15_time',
       'trial_15_winner', 'trial_16_time', 'trial_16_winner', 'trial_17_time',
       'trial_17_winner', 'trial_18_time', 'trial_18_winner', 'trial_19_time',
       'trial_19_winner', 'trial_20_time', 'trial_20_winner', 'mouse_1_wins',
       'mouse_2_wins', 'ties', 

In [101]:
sheet_name_to_everything[key]["processed_behavior_recording_dataframe"]

,date,cage,box,match,scorer,trial_1_time,trial_1_winner,trial_2_time,trial_2_winner,trial_3_time,...,trial_20_time,trial_20_winner,mouse_1_wins,mouse_2_wins,ties,sheet_name,session_number_difference,,processed_cage_num_of_winner,processed_cage_num_of_loser
0,2022-05-19,1,1.0,1.1 v 1.2,NaN,1:12-1:22,1.1,2:31-2:41,1.1,4:00-4:10,...,29:53-30:03,1.1,11.0,0,0.0,Master Master,NaN,Master Master,Master Master,Master Master
1,2022-05-19,1,2.0,1.3 v 1.4,NaN,1:12-1:22,1.4,2:30-2:40,1.4,3:59-4:09,...,29:53-30:03,1.3,0.0,9,0.0,Master Master,0.0,Master Master,Master Master,Master Master
2,2022-05-19,1,3.0,1.1 v 1.3,NaN,1:07-1:17,1.3,3:56-4:06,tie,5:14-5:24,...,29:53-30:03,1.3,0.0,9,0.0,Master Master,0.0,Master Master,Master Master,Master Master
3,2022-05-19,1,4.0,1.2 v 1.4,NaN,1:06-1:16,1.4,2:25-2:35,1.2,3:54-4:04,...,29:48-29:58,1.4,0.0,checked whole row,0.0,Master Master,0.0,Master Master,Master Master,Master Master
4,2022-05-19,1,2.0,1.2 v 1.3,NaN,1:11-1:21,1.2,2:30-2:40,1.3,4:00-4:10,...,29:53-30:03,1.2,0.0,checked whole row,0.0,Master Master,0.0,Master Master,Master Master,Master Master
5,2022-05-19,1,1.0,1.1 v 1.4,NaN,1.12-1.22,1.4,1.30-1.40,1.1,4.00-4.10,...,29.53-30.03,1.1,0.0,checked whole row,0.0,Master Master,0.0,Master Master,Master Master,Master Master
6,2022-05-20,1,4.0,1.1 v 1.4,NaN,1:08-1:18,1.1,2:28-2:38,1.1,3:56-4:06,...,NO TRIAL,NO TRIAL,16.0,2,1.0,Master Master,1.0,Master Master,Master Master,Master Master
7,2022-05-20,1,3.0,1.2 v 1.3,NaN,1:10-1:20,1.2,2:28-2:38,1.2,3:57-4:07,...,NO TRIAL,NO TRIAL,16.0,2,1.0,Master Master,0.0,Master Master,Master Master,Master Master
8,2022-05-20,1,2.0,1.1 v 1.3,NaN,1:05-1:15,1.1,2:24-2:34,tie,3:53-4:03,...,29:47-29:57,1.1,16.0,checked whole row,1.0,Master Master,0.0,Master Master,Master Master,Master Master
9,2022-05-20,1,1.0,1.2 v 1.4,NaN,1:05-1:15,1.2,2:26-2:36,1.4,3:52-4:02,...,29:47-29:57,1.1,16.0,checked whole row,1.0,Master Master,0.0,Master Master,Master Master,Master Master


## Calculating Elo rating

- Example calculation

In [102]:
calculation.calculate_elo_rating(subject_elo_rating=1000, agent_elo_rating=2000, score=1)

1019.9

In [103]:
calculation.update_elo_rating(winner_id="A", loser_id="B")

defaultdict(<function elorating.calculation.update_elo_rating.<locals>.<lambda>()>,
            {'A': 1010.0, 'B': 990.0})

## Get the Elo rating for all the events

- Going through each row or interaction and calculating the new Elo rating for the winner and loser. This will create a new dataframe based off of the processed behavioral recording dataframe

# NOTE: If there are a set of columns to keep, edit the cell below with the name of the columns each in quotation marks seperated by commas
   - i.e. `['runner', 'date', 'match', 'winner', 'loser', 'notes', 'session_number',
       'session_number_difference']`

In [108]:
all_columns_to_keep = []

In [109]:
def get_subject_cage_number(x):
    """
    """
    if x["win_draw_loss"] == 1:
        return x["processed_cage_num_of_winner"]
    elif x["win_draw_loss"] == 0:
        return x["processed_cage_num_of_loser"]
    elif x["win_draw_loss"] == 0.5:
        if 1 == pairing_index:
            return x["processed_cage_num_of_winner"] 
        elif 1 == pairing_index:
            return x["processed_cage_num_of_loser"]

In [110]:
copy_sheet_name_to_everything = copy.copy(sheet_name_to_everything)
for key, value in sheet_name_to_everything.items():
    try:
        if not all_columns_to_keep:
            # Asking the user which columns to carry over to the Elo rating calculation dataframe
            value["columns_to_keep_string"] = input("""Type all the columns that will be copied from the original dataframe to the Elo rating calculated dataframe. 

            All the available actions: {}
            Each column must be put in quotes and seperated by a comma(,). i.e. 'length of observations', 'date', 'cage #'

            NOTE: If left blank, all the columns will be kept
            """.format(value["processed_behavior_recording_dataframe"].columns))


            # Making a list out of the string of inputted sheet names
            if value["columns_to_keep_string"]:
                value["columns_to_keep_string"] = "[" + value["columns_to_keep_string"] + "]"
                value["columns_to_keep_list"] = ast.literal_eval(value["columns_to_keep_string"])
            else:
                value["columns_to_keep_list"] = value["processed_behavior_recording_dataframe"].columns
        else: 
            value["columns_to_keep_list"] = all_columns_to_keep

        try:
            # Calculating the Elo rating    
            value["index_to_elo_rating_and_meta_data"] = calculation.iterate_elo_rating_calculation_for_dataframe(dataframe=value["processed_behavior_recording_dataframe"], \
                                                                                                                       winner_id_column=value["winner_id_column"], loser_id_column=value["loser_id_column"], \
                                                                                                                                 additional_columns=value["columns_to_keep_list"], tie_column=value["ties_column"])

        except:
            nonexistent_columns = set(value["columns_to_keep_list"]) - set(value["processed_behavior_recording_dataframe"].columns)
            print("WARNING: {} does not have {} columns".format(key, nonexistent_columns))
            warnings.warn("Look at warning from above or below")
            # Using all the column names if no column name is specified
            # Removing the winner and loser column because they will be specified as the columns for the Elo rating calculation
            value["columns_to_keep_list"] =  sorted(list(set(value["processed_behavior_recording_dataframe"].columns) - set([value["winner_id_column"]]) - set([value["loser_id_column"]])))
            value["index_to_elo_rating_and_meta_data"] = calculation.iterate_elo_rating_calculation_for_dataframe(dataframe=value["processed_behavior_recording_dataframe"], \
                                                                                                                       winner_id_column=value["winner_id_column"], loser_id_column=value["loser_id_column"], \
                                                                                                                                 additional_columns=value["columns_to_keep_list"], tie_column=value["ties_column"])     

        # Making a dataframe from the dictionary 
        value["elo_rating_dataframe"] = pd.DataFrame.from_dict(value["index_to_elo_rating_and_meta_data"], orient="index")
        try:
            value["elo_rating_dataframe"]["cage_num_of_subject"] = value["elo_rating_dataframe"].apply(lambda x: get_subject_cage_number(x), axis = 1)
            value["elo_rating_dataframe"]["cage_num_of_agent"] = value["elo_rating_dataframe"].apply(lambda x: get_subject_cage_number(x), axis = 1)
        except:
            value["elo_rating_dataframe"]["cage_num_of_subject"] = key
            value["elo_rating_dataframe"]["cage_num_of_agent"] = key
    except:
        copy_sheet_name_to_everything.pop(key)
sheet_name_to_everything = copy_sheet_name_to_everything

In [ ]:
raise ValueError()

In [107]:
value["elo_rating_dataframe"]

KeyError: 'elo_rating_dataframe'

In [ ]:
value["elo_rating_dataframe"].head()

In [ ]:
value["elo_rating_dataframe"].tail()

In [ ]:
value["elo_rating_dataframe"].groupby("subject_id").count()

## Combining all the Elo rating dataframes into one

In [ ]:
# Putting all the dataframes into one list
all_sheet_elo_scord_dataframe_list = []
for key, value in sheet_name_to_everything.items():    
    all_sheet_elo_scord_dataframe_list.append(value["elo_rating_dataframe"])

# Combining all the dataframes into one
all_sheet_elo_scord_dataframe_combined = pd.concat(all_sheet_elo_scord_dataframe_list)

In [ ]:
all_sheet_elo_scord_dataframe_combined

- Adding the strain information

In [ ]:
all_subject_ids = set(all_sheet_elo_scord_dataframe_combined["subject_id"].unique()).union(set(all_sheet_elo_scord_dataframe_combined["agent_id"].unique()))

# NOTE: If there are strains that are associated to each cage, then create a dictionary of cage numbers to strains inside the `{}`
- i.e. `cage_to_strain = {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6": "CD1"}`

In [ ]:
cage_to_strain = {}

In [ ]:
if cage_to_strain:
    all_sheet_elo_scord_dataframe_combined["subject_strain"] = all_sheet_elo_scord_dataframe_combined["cage_num_of_subject"].map(cage_to_strain)
    all_sheet_elo_scord_dataframe_combined["agent_strain"] = all_sheet_elo_scord_dataframe_combined["cage_num_of_agent"].map(cage_to_strain)

- Adding the name of the experiment

In [ ]:
# Adding the name of the experiment
all_sheet_elo_scord_dataframe_combined["experiment_type"] = protocol_name

- Adding the cohort

In [ ]:
all_sheet_elo_scord_dataframe_combined["cohort"] = cohort_name

In [ ]:
all_sheet_elo_scord_dataframe_combined.head()

In [ ]:
all_sheet_elo_scord_dataframe_combined.tail()

In [ ]:
# Checking to see how many rows for each subject in each cage
all_sheet_elo_scord_dataframe_combined.groupby(['subject_id','cage_num_of_subject']).count()

## Creating a dataframe with only the final Elo rating for each subject

In [ ]:
# Checking to see which cage and subject combination has more than one row
all_sheet_elo_scord_dataframe_groupby = all_sheet_elo_scord_dataframe_combined.groupby(['subject_id','cage_num_of_subject']).size().reset_index()
all_sheet_elo_scord_dataframe_groupby = all_sheet_elo_scord_dataframe_groupby[all_sheet_elo_scord_dataframe_groupby[0] >= 1]

# Going through each combination and saving the combination to a dictionary
index_to_subject_id_and_processed_cage_number = defaultdict(dict)
for index, row in all_sheet_elo_scord_dataframe_groupby.iterrows():
    index_to_subject_id_and_processed_cage_number[index]['subject_id'] = row['subject_id']
    index_to_subject_id_and_processed_cage_number[index]['cage_num_of_subject'] = row['cage_num_of_subject']


In [ ]:
index_to_subject_id_and_processed_cage_number

- Getting the final Elo rating for each cage and subject combination

In [ ]:
for index, (key, value) in enumerate(index_to_subject_id_and_processed_cage_number.items()):   
    # The Elo rating dataframe for each cage and subject combination
    per_subject_dataframe = all_sheet_elo_scord_dataframe_combined[(all_sheet_elo_scord_dataframe_combined["subject_id"] == value["subject_id"]) & (all_sheet_elo_scord_dataframe_combined["cage_num_of_subject"] == value["cage_num_of_subject"])]
    # Getting the final Elo rating for each combination
    # -1 Means that we're getting the data from the last row

    index_to_subject_id_and_processed_cage_number[index]["final_elo_rating"] = per_subject_dataframe.iloc[-1]["updated_elo_rating"]
    index_to_subject_id_and_processed_cage_number[index]["cohort"] = per_subject_dataframe.iloc[-1]["cohort"]
    try:
        index_to_subject_id_and_processed_cage_number[index]["strain"] = per_subject_dataframe.iloc[-1]["strain"]
    except:
        print("WARNING: {} in cage {} does not have strain information".format(index_to_subject_id_and_processed_cage_number[key]["subject_id"], index_to_subject_id_and_processed_cage_number[key]["cage_num_of_subject"]))
        warnings.warn("Look at warning from above or below")


In [ ]:
id_to_final_elo_rating_df = pd.DataFrame.from_dict(index_to_subject_id_and_processed_cage_number, orient="index")
# Adding protocol name
id_to_final_elo_rating_df["experiment_type"] = protocol_name
# Adding rank
id_to_final_elo_rating_df["rank"] = id_to_final_elo_rating_df.groupby("cage_num_of_subject")["final_elo_rating"].rank("dense", ascending=False)
# Sorting by cage and then id
id_to_final_elo_rating_df = id_to_final_elo_rating_df.sort_values(by=['cage_num_of_subject', "subject_id"], ascending=True).reset_index(drop=True)

In [ ]:
id_to_final_elo_rating_df.head()

In [ ]:
id_to_final_elo_rating_df.tail()

# Making plots for all sheets

- Getting the dates the files were being recorded to use for the file name

In [ ]:
all_sheet_elo_scord_dataframe_combined.head()

In [ ]:
all_sheet_elo_scord_dataframe_combined.tail()

- Getting the earliest and the latest dates for all the recordings

In [ ]:
all_earlist_dates = []
all_latest_dates = []
for key, value in sheet_name_to_everything.items():
    try:
        # Getting all the earliest dates for each sheet
        all_earlist_dates.append(value["elo_rating_dataframe"][value["session_divider_column"]].min())
        all_latest_dates.append(value["elo_rating_dataframe"][value["session_divider_column"]].max())
    except:
        print("WARNING: {} does not have dates as columns".format(key))
        warnings.warn("Look at warning from above or below")

In [ ]:
try:
    # Turning the Dates into a easier to read format
    # Getting the 0th part of split to remove seconds
    earliest_date = str(min(all_earlist_dates)).split()[0]
    latest_date = str(max(all_latest_dates)).split()[0]
    print("Earlist date: {}".format(earliest_date))
    print("Latest date: {}".format(latest_date))
except:
    earliest_date = None
    latest_date = None

- Getting the cage numbers

In [ ]:
all_cages_list = []
# Creating a list of all the cage numbers
for key, value in sheet_name_to_everything.items():
    try:
        for cage in value["elo_rating_dataframe"]["cage_num_of_subject"].unique():
            all_cages_list.append(cage)
    except:
        print("WARNING: {} does not have cage number as columns".format(key))
        warnings.warn("Look at warning from above or below")

In [ ]:
try:
    all_cages_string = "-".join(sorted([sheet.lower().strip("cage").strip() for sheet in all_cages_list]))
    all_cages_string = "cages-{}".format(all_cages_string)
    print("String of cage names to use for file name: {}".format(all_cages_string))
except: 
    warnings.warn("WARNING: There are no cage numbers to make a title out of")
    all_cages_string = None

- Creating an output directory to save the plots

In [ ]:
plot_output_directory = os.path.join(".", "proc", "plots", "{}_elo_rating".format(protocol_name))

In [ ]:
plot_output_directory

In [ ]:
os.makedirs(plot_output_directory, exist_ok=True)

# **NOTE**: Sometimes this cell needs to be run again to make sure the size is correct

In [ ]:
# Getting the highest and lowest Elo rating for cutoffs of the Y-axis
max_elo_rating = all_sheet_elo_scord_dataframe_combined["updated_elo_rating"].max()
min_elo_rating = all_sheet_elo_scord_dataframe_combined["updated_elo_rating"].min()

plt.rcParams["figure.figsize"] = (13.5,7.5)
# Making a plot for each sheet
for key, value in sheet_name_to_everything.items():
    # Setting the size of the figure
    plt.rcParams["figure.figsize"] = (13.5,7.5)
    print(key)
    elo_rating_dataframe = value["elo_rating_dataframe"]
    # Using a new figure template for each sheet
    fig, ax = plt.subplots()        
        
    try:
        # Drawing vertical lines that represent when each session begins
        # Based on when a row has a different session than the previous row
        for index, row in elo_rating_dataframe[elo_rating_dataframe['session_number_difference'].astype(bool)].iterrows():
            # Offsetting by 0.5 to avoid drawing the line on the dot
            # Drawing the lines a little above the max and a little below the minimum
            plt.vlines(x=[row["total_match_number"] - 0.5], ymin=min_elo_rating-50, ymax=max_elo_rating+50, colors='black', linestyle='dashed')
    except:
        print("WARNING: {} does not have a column for session divider".format(key))
        warnings.warn("Look at warning from above or below")
            
    # Drawing a line for each subject
    for subject in sorted(elo_rating_dataframe["subject_id"].unique()):
        # Getting all the rows with the current subject
        subject_dataframe = elo_rating_dataframe[elo_rating_dataframe["subject_id"] == subject]
        # Making the current match number the X-Axis
        plt.plot(subject_dataframe["total_match_number"], subject_dataframe["updated_elo_rating"], '-o', label=subject)

    # Labeling the X/Y Axis and the title
    ax.set_xlabel("Trial Number")
    ax.set_ylabel("Elo rating")
    # Formattnig Cohort and Experiment Name so that it's more readable with spacing and capitalization
    try:
        formatted_cohort_name = " ".join(cohort_name.split("_")).capitalize()
    except:
        formatted_cohort_name = cohort_name
    try:
        formatted_protocol_name = string.capwords(" ".join(protocol_name.split("_")))
    except:
        formatted_protocol_name = protocol_name
    try:
        formatted_cage_name = " ".join((re.match(r"([a-z]+)([0-9]+)", key, re.I).groups())).capitalize()    
    except:
        formatted_cage_name = key
    ax.set_title("{} Elo Rating for {} {}".format(formatted_protocol_name, formatted_cohort_name, formatted_cage_name))
    
    # To show the legend
    ax.legend(loc="upper left")
    # Setting the values of the Y-axis
    plt.ylim(min_elo_rating-50, max_elo_rating+50) 
    # Saving the plot
    file_name_parts_separated = [prefix_name, cohort_name, key, earliest_date, latest_date]
    file_name_parts_combined = "_".join([part for part in file_name_parts_separated if part])
    
    file_name_full = "elo_rating_{}.png".format(file_name_parts_combined)
    # Removing all the spaces and replacing them with underscores
    file_name_full = "_".join(file_name_full.split(" "))
    plt.savefig(os.path.join(plot_output_directory, file_name_full))
    # Showing the plots
    plt.show()

# Saving the Dataframes

- Saving the dataframes to a file

In [ ]:
file_name_parts_separated = [cohort_name, all_cages_string, prefix_name, earliest_date, latest_date]
file_name_parts_combined = "_".join([part for part in file_name_parts_separated if part])

file_name_full = "{}_elo-rating-history.csv".format(file_name_parts_combined)
print(file_name_full)
all_sheet_elo_scord_dataframe_combined.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name_full))

In [ ]:
file_name_full = "{}_final-elo-rating.csv".format(file_name_parts_combined)
print(file_name_full)
id_to_final_elo_rating_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name_full))

# Seeing which subject is the dominant or submissive

- Grouping all the rows with the same pair

In [ ]:
all_processed_behavior_recording_list = []
for key, value in sheet_name_to_everything.items():
    all_processed_behavior_recording_list.append(value["processed_behavior_recording_dataframe"])
    

- Combining all the dataframes from all the cages

In [ ]:
all_processed_behavior_recording_df = pd.concat(all_processed_behavior_recording_list)

In [ ]:
all_processed_behavior_recording_df.head()

- Getting a tuple of the animal IDs to be able to group

# Note: This assumes that all the animal IDs are different across cages and that all IDs are numbers. i.e. there are no duplicate IDs in different cages.

In [ ]:
# Getting the animal IDs from the Match string
# i.e. Getting all the floats and removing all spaces
# Sorting so that the animal IDs are always in the same order
all_processed_behavior_recording_df["animal_id"] =  all_processed_behavior_recording_df["match"].apply(lambda x: sorted([subject_id.lower().strip() for subject_id in re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)]))


In [ ]:
# Making a tuple out of the list
# Tuples are used because lists are mutable and can't be grouped with
all_processed_behavior_recording_df["tuple_animal_id"] = all_processed_behavior_recording_df["animal_id"].apply(lambda x: tuple(x))

In [ ]:
all_processed_behavior_recording_df.head()

- Removing columns that would be unnecessary for the pairings

In [ ]:
all_processed_behavior_recording_df.columns

In [ ]:
# Getting only the columns that we need
all_processed_behavior_recording_df = all_processed_behavior_recording_df[['date', 'winner', 'loser', 'animal_id', 'tuple_animal_id', "processed_cage_num_of_winner", "processed_cage_num_of_loser"]]

In [ ]:
all_processed_behavior_recording_df.head()

- Getting the ID of the winner and the loser for each pair with each match

In [ ]:
all_wins_per_pair = all_processed_behavior_recording_df.groupby("tuple_animal_id")['winner'].apply(list)
all_loses_per_pair = all_processed_behavior_recording_df.groupby("tuple_animal_id")['loser'].apply(list)

In [ ]:
all_wins_per_pair[:5]

- Making a dataframe of all the winner IDs and all the loser IDs for a given pair

In [ ]:
all_competition_per_pair_df = pd.concat([all_wins_per_pair, all_loses_per_pair], axis=1).reset_index()

In [ ]:
all_competition_per_pair_df = all_competition_per_pair_df.rename(columns={k: prefix_name + "_" + k for k in all_competition_per_pair_df.columns})

In [ ]:
all_competition_per_pair_df

- Adding the cage information

In [ ]:
# Getting the cage number for each pair
dropped_duplicate_all_processed_behavior_recording_df = all_processed_behavior_recording_df[["tuple_animal_id"]].drop_duplicates()

In [ ]:
dropped_duplicate_all_processed_behavior_recording_df.head()

- Calculating the overall winner and loser. Also seeing if there is signficant difference in the number of wins to see if one is dominant over the other

In [ ]:
all_competition_per_pair_df["cohort"] = cohort_name


In [ ]:
all_competition_per_pair_df["{}_averaged_winner".format(prefix_name)] = all_competition_per_pair_df["{}_winner".format(prefix_name)].apply(lambda x: Counter(x).most_common(1)[0][0])
all_competition_per_pair_df["{}_averaged_loser".format(prefix_name)] = all_competition_per_pair_df["{}_loser".format(prefix_name)].apply(lambda x: Counter(x).most_common(1)[0][0])
all_competition_per_pair_df["{}_winner_count".format(prefix_name)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner".format(prefix_name)].count(x["{}_averaged_winner".format(prefix_name)]), axis=1)
all_competition_per_pair_df["{}_loser_count".format(prefix_name)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner".format(prefix_name)].count(x["{}_averaged_loser".format(prefix_name)]), axis=1)
all_competition_per_pair_df["{}_count_difference".format(prefix_name)] = all_competition_per_pair_df["{}_winner_count".format(prefix_name)] - all_competition_per_pair_df["{}_loser_count".format(prefix_name)]
all_competition_per_pair_df["{}_match_count".format(prefix_name)] = all_competition_per_pair_df["{}_winner".format(prefix_name)].apply(lambda x: len(x))
all_competition_per_pair_df["{}_percent_win".format(prefix_name)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner_count".format(prefix_name)] / x["{}_match_count".format(prefix_name)], axis=1)
all_competition_per_pair_df["{}_percentage_tie".format(prefix_name)] = all_competition_per_pair_df["{}_percent_win".format(prefix_name)].apply(lambda x: True if x < 0.75 else False)

In [ ]:
all_competition_per_pair_df

- Saving the competiton pair results dataframe to a file

In [ ]:
file_name = "{}_{}_grouped_by_pairs_cage_{}_date_{}_{}.csv".format(cohort_name, prefix_name, all_cages_string, earliest_date, latest_date)


In [ ]:
elo_rating_spreadsheet_output_directory

In [ ]:
file_name

In [ ]:
all_competition_per_pair_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name))


# TODO

# Reward Competition Elo Score Calculation

In [ ]:
import re
import os
import sys
import string
from collections import defaultdict
import glob
import warnings
from datetime import datetime
from datetime import timedelta
import ast
from collections import Counter

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xlrd

In [ ]:
# setting path
sys.path.append('../../../src')

In [ ]:
from eloscore import eloscore

In [ ]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

## TODO: ADD Summary

# Make it iterate through a list of sheets

- Notebook that calcuates the Elo Score of mice competing over access to a tone associated reward port
- The data is a spreadsheet of combination of subjects (as rows) against dates (as columns)

## Name of protocol for naming

- This name will be used to name files and title plots. Please change if you are using a different protocol or adding more details
    - **NOTE**: This should be changed based on the name the protocol

In [ ]:
protocol_name = "reward_competition"

In [ ]:
cohort_name = "pilot_3"

## Getting the file name of the raw data

- Default input folder and keyword to search the files for 
    - **NOTE**: This should not be changed unless there is a consistent change with the file naming convention

In [ ]:
input_folder = "./data"

In [ ]:
# Accomodates for both capitalization of the file names
raw_data_file_keyword = '*[rR]eward*'

# NOTE: If there is a set excel file used for this notebook, edit the cell below with the path to that in the "" (quotation marks)

In [ ]:
raw_data_file_path = "./data/Reward_Comp_Scoring_Assignments (1).xlsx"

- Asking the user what the path to the recording files are, with the option of using wildcards

In [ ]:
if not raw_data_file_path:
    raw_data_glob_pattern = input("""Type out the path(address) of the raw behavioral recording excel sheets.
    Remember that if you are using a relative path, it will be based off of the location of this Jupyter Notebook.

    Globbing can also be used if you want to search with a wild card(Capitalization matters). 
    i.e. './data/*Home*' will be able to find './data/Homecage_observations.xlsx'

    NOTE: If left blank, the path will automatically be guessed off of the default settings 
    """)

- By default, this will search for files that are in the `./data` folder (in the folder that this notebook is in) that have key word you specified in the file name

In [ ]:
if not raw_data_file_path:

    # Using the user inputted path/pattern
    if raw_data_glob_pattern.strip():
        # Getting a list of all matching files
        raw_data_files_list = glob.glob(raw_data_glob_pattern.strip())
        # Checking if there were any files that matched
        if raw_data_files_list:
            raw_data_file_path = raw_data_files_list[0]
        else:
            raise ValueError("No files were found with the path/pattern of {}. Please rerun the previous cell with the correct path".format(raw_data_glob_pattern))

    # Using the default pattern
    else:
        # Searching for matching files with recursion
        if glob.glob(os.path.join(input_folder + "**", raw_data_file_keyword)):
            raw_data_glob_pattern = os.path.join(input_folder + "**", raw_data_file_keyword)
            # Getting a list of all matching files
            raw_data_files_list = glob.glob(raw_data_glob_pattern)
        # Searching for matching files only in the specified folder
        elif glob.glob(os.path.join(input_folder, raw_data_file_keyword)):
            raw_data_glob_pattern = glob.glob(os.path.join(input_folder, raw_data_file_keyword))
            # Getting a list of all matching files
            raw_data_files_list = glob.glob(raw_data_glob_pattern)
        else:
            raise ValueError("No files were found in {} that had the keyword {} in it".format(input_folder, raw_data_file_keyword))


- Checking to see if only one file is specified

In [ ]:
if not raw_data_file_path:

    if len(raw_data_files_list) >= 2:
        raise ValueError("More than one file was found with the path/pattern of {}. Please rerun the previous cell with the correct path".format(raw_data_glob_pattern))
    else:
        # Using the first(and only file) as the file path
        raw_data_file_path = raw_data_files_list[0]

In [ ]:
raw_data_file_path

## Getting a list of all the sheets

In [ ]:
# Getting the sheet names for the excel file
xls = pd.ExcelFile(raw_data_file_path)
raw_data_sheet_names = xls.sheet_names

In [ ]:
raw_data_sheet_names

# NOTE: If there is a set sheet names used for this notebook, edit the cell below with the name of the sheets each in quotation marks seperated by commas
   - i.e. `['CAGE 1', 'CAGE 2', 'CAGE 3', 'CAGE 4']`

In [ ]:
inputted_sheet_names_list = ['Master List']

- Asking the user what sheets they want to use

In [ ]:
if not inputted_sheet_names_list:

    inputted_sheet_names_string = input("""Type out the name of the sheets that you want to be processed. 
    Each name must be put in quotes and seperated by a comma(,). i.e. "CAGE3", "CAGE4"

    The available sheets are: {}

    Alternatively, you can use the index of the list of names above. 
    Remember, that Python is zero indexed so the first item will be have the 0 index, second the 1 index, and so on. 
    i.e. 1, 2

    NOTE: If left blank, all sheets will be used
    """.format(raw_data_sheet_names))

In [ ]:
if not inputted_sheet_names_list:

    # Making a list out of the string of inputted sheet names
    if inputted_sheet_names_string:
        inputted_sheet_names_string = "[" + inputted_sheet_names_string + "]"
        # Turning the string into a list
        inputted_sheet_names_list = ast.literal_eval(inputted_sheet_names_string)
    # Using all the sheet names if no sheet name is specified
    else:
        inputted_sheet_names_list =  raw_data_sheet_names

In [ ]:
inputted_sheet_names_list

- Converting all the numbers into the sheet name that the index of the number corresponds to

In [ ]:
if not inputted_sheet_names_list:

    for index, sheet in enumerate(inputted_sheet_names_list):
        # Checking if the sheet name was a number
        if isinstance(sheet, int) and str(sheet).isdigit():
            inputted_sheet_names_list[index] =  raw_data_sheet_names[sheet]

In [ ]:
inputted_sheet_names_list

- Checking to see if all the sheets are in the excel file

In [ ]:
if not set(inputted_sheet_names_list).issubset(raw_data_sheet_names):
    # Getting all the sheets that were not in the original spreadsheet
    not_included_sheet_names = set(inputted_sheet_names_list) - set(raw_data_sheet_names)
    raise ValueError("All the listed sheet names are not in {}".format(not_included_sheet_names))

# NOTE: If there is a set row that has the column names, then enter the row number in the quotation marks

In [ ]:
# Making the row number a string, so that "0" is treated as a True value
all_header_row = header_row = "0"
header_row = int(header_row)

## Finding the row for the header

- Headers are the row in a spreadsheet that has all the column names. Sometimes spreadsheets don't use the first row as the row with the column names. So, the row for the header will be asked or assumed to be the first row

In [ ]:
sheet_name_to_everything = defaultdict(dict)
for sheet in inputted_sheet_names_list:
    if not all_header_row:
        print("\nCurrent Sheet Name: {}".format(sheet))    
        per_sheet_dataframe = pd.read_excel(raw_data_file_path, sheet_name=sheet, header=0)

        # Showing the columns that are chosen with the header being the 0th row
        print("Columns Names: {}".format(per_sheet_dataframe.columns))
        # Show the dataframe that would be created with the header being the 0th row
        print("First few rows of this dataframe:")
        print(pd.read_excel(raw_data_file_path, sheet_name=sheet, header=0).head())

        # Allowing the user the choose the row number for the header
        header_row = input("""Type the row number to be used as the header
        (AKA the row with the column name that you want to use.)
        If you want to keep the column names that were displayed, type 0.
        If you want to use a different row, then type the corresponding number. 

        The rows displayed in this cell are dataframes created from Pandas. 
        To use the row with the 0 index for column names, type 1. 
        For the row with the 1 index, it will be 2 and so on. i.e. 2

        If you are looking at the original spread sheet, remember that Python is zero indexed. 
        So the first row will be 0, second will be 1, and so on. 
        i.e. 1

        NOTE: If left blank, the original row that was used will be used.
        """).strip()

        if header_row == "":
            header_row = 0
        else:
            header_row = int(header_row)
    
    # Checking if any of the column names are from empty cells
    column_names = "".join([str(col) for col in pd.read_excel(raw_data_file_path, sheet_name=sheet, header=header_row).columns])
    # If a column name came from an empty cell, it would have "Unnamed" in it
    if "Unnamed" in column_names:
        raise ValueError("""Not all the cells in the chosen row are filled in.
                         Please choose a row that has the name of the columns
                         
                         The values in this row are: {}""".format(column_names))
    else:
        sheet_name_to_everything[sheet]["header_row"] = header_row

In [ ]:
sheet_name_to_everything

## Reading in all the spreadsheets

In [ ]:
# Going through each sheet and creating a dataframe of it
for key, value in sheet_name_to_everything.items():
    sheet_name_to_everything[key]["original_behavior_recording_dataframe"] = pd.read_excel(raw_data_file_path, sheet_name=key, header=value["header_row"])

In [ ]:
sheet_name_to_everything[key]["original_behavior_recording_dataframe"].head()

In [ ]:
sheet_name_to_everything[key]["original_behavior_recording_dataframe"].tail()

## Standarizing the Columns

- Making all the column names lower case and removing any extra spaces in the beginning and at the end

In [ ]:
for key, value in sheet_name_to_everything.items():
    # Creating a dictionary that maps the original column name to the standarized one
    column_name_to_standarized = defaultdict(dict)
    for col in sheet_name_to_everything[key]["original_behavior_recording_dataframe"]:
        # Making the column name lower case and removing the spaces
        column_name_to_standarized[col] = str(col).lower().strip()
    sheet_name_to_everything[key]["column_name_to_standarized"] = column_name_to_standarized

In [ ]:
sheet_name_to_everything[key]["column_name_to_standarized"]

In [ ]:
# Renaming all the columns to the lower case and space removed version
for key, value in sheet_name_to_everything.items():
    sheet_name_to_everything[key]["processed_behavior_recording_dataframe"] = sheet_name_to_everything[key]["original_behavior_recording_dataframe"].rename(columns=sheet_name_to_everything[key]["column_name_to_standarized"])

In [ ]:
sheet_name_to_everything[key]["processed_behavior_recording_dataframe"].head()

## Removing Unnecessary Columns

# TODO: Pick which columns to drop or keep

In [ ]:
reward_competition_df = sheet_name_to_everything[key]["processed_behavior_recording_dataframe"]

In [ ]:
reward_competition_df.columns

In [ ]:
reward_competition_df = reward_competition_df.drop(["mouse 1 wins", "mouse 2 wins", "ties", " "], axis=1, errors="ignore")

In [ ]:
columns_to_keep = [col for col in reward_competition_df.columns if "time" not in col.lower().strip()]

In [ ]:
columns_to_keep

In [ ]:
reward_competition_df = reward_competition_df[columns_to_keep].copy()

In [ ]:
reward_competition_df.head()

In [ ]:
reward_competition_df = reward_competition_df.dropna(subset=["trial 1 winner"])

## Add both IDs as a column

# TODO: Pick the match column

In [ ]:
reward_competition_df["animal_ids"] = reward_competition_df["match"].apply(lambda x: tuple(sorted([all_ids.strip() for all_ids in re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)])))

In [ ]:
reward_competition_df.head()

In [ ]:
reward_competition_df["cohort"] = cohort_name

In [ ]:
cage_to_strain = {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6": "CD1"}

# NOTE: If there are strains that are associated to each cage, then create a dictionary of cage numbers to strains inside the `{}`
- i.e. `cage_to_strain = {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6": "CD1"}`

In [ ]:
reward_competition_df["strain"] = reward_competition_df["cage"].astype(str).map(cage_to_strain)

In [ ]:
all_cages = "_".join([str(cage) for cage in sorted(reward_competition_df["cage"].unique())])

In [ ]:
all_cages

In [ ]:
earliest_date = reward_competition_df["date"].min()
latest_date = reward_competition_df["date"].max()

In [ ]:
earliest_date = str(earliest_date.date()).replace("-", "")

In [ ]:
latest_date = str(latest_date.date()).replace("-", "")

In [ ]:
earliest_date

In [ ]:
latest_date

In [ ]:
file_name = "{}_{}_trial_as_columns_cages_{}_date_{}_{}.csv".format(cohort_name, protocol_name, all_cages, earliest_date, latest_date)

reward_competition_df.to_csv(os.path.join("./proc/elo_score_spread_sheets/reward_competition_elo_score/", file_name))

## Melt 

In [ ]:
reward_competition_df["index"] = reward_competition_df.index
reward_competition_df = reward_competition_df.reset_index(drop=True)

In [ ]:
reward_competition_df.head()

# TODO: Pick the columns to keep

In [ ]:
melted_reward_competition_df = reward_competition_df.melt(id_vars=["index", "date", "cage", "box", "match", "scorer", "animal_ids"], 
        var_name="trial", 
        value_name="winner")

In [ ]:
melted_reward_competition_df.head()

- Dropping all rows that don't contain Winner

In [ ]:
melted_reward_competition_df = melted_reward_competition_df.dropna(subset="winner")

- Making all the ids into string

In [ ]:
melted_reward_competition_df["winner"] = melted_reward_competition_df["winner"].astype(str)
melted_reward_competition_df["winner"] = melted_reward_competition_df["winner"].apply(lambda x: x.lower().strip())

In [ ]:
melted_reward_competition_df["match"].unique()

- Making a different column for ties

In [ ]:
melted_reward_competition_df["tie"] = melted_reward_competition_df["winner"].apply(lambda x: True if "tie" in x.lower().strip() else False)

- Replacing tie with the first animal id for elo score plotting

In [ ]:
melted_reward_competition_df["winner"] = melted_reward_competition_df.apply(lambda x: x["animal_ids"][0] if x["tie"] else x["winner"], axis=1)

In [ ]:
melted_reward_competition_df[melted_reward_competition_df["tie"]]

- Dropping all non tie and competition rows

In [ ]:
melted_reward_competition_df["winner_is_number"] = melted_reward_competition_df["winner"].apply(lambda x: True if re.match(r'^-?\d+(?:\.\d+)$', x) else False)


In [ ]:
melted_reward_competition_df = melted_reward_competition_df[melted_reward_competition_df["winner_is_number"] == True]
melted_reward_competition_df = melted_reward_competition_df.drop(columns=["winner_is_number"])

In [ ]:
melted_reward_competition_df.head()

- Getting the number of the trial so that we can order by number(instead of string, which would make 11 come before 2)

In [ ]:
melted_reward_competition_df["trial_number"] = melted_reward_competition_df["trial"].apply(lambda x: int(x.lower().strip("trial").strip("winner")))

In [ ]:
melted_reward_competition_df = melted_reward_competition_df.sort_values(["index", "trial_number"])

In [ ]:
melted_reward_competition_df.head()

In [ ]:
melted_reward_competition_df["trial_number"].unique()

## Add a column for the loser ID

In [ ]:
melted_reward_competition_df.columns

In [ ]:
melted_reward_competition_df.head()

In [ ]:
melted_reward_competition_df["loser"] = melted_reward_competition_df.apply(lambda x: (list(set(x["animal_ids"]) - set([x["winner"]]))[0]), axis=1)

In [ ]:
melted_reward_competition_df["loser"].unique()

In [ ]:
melted_reward_competition_df.head(n=5)

In [ ]:
melted_reward_competition_df.tail(n=5)

## Calculating ELO Score

- Example calculation

In [ ]:
eloscore.calculate_elo_score(subject_elo_score=1000, agent_elo_score=2000)

# TODO: Find a way to account for ties

## Get the Elo score for all the events

In [ ]:
eloscore.update_elo_score(winner_id="A", loser_id="B")

- Function that creates a dictionary that has the original/updated elo score for each event

In [ ]:
cage_to_elo_score_dict = defaultdict(dict)
for cage in melted_reward_competition_df["cage"].unique():
    cage_df = melted_reward_competition_df[melted_reward_competition_df["cage"] == cage]
    cage_to_elo_score_dict[cage] = eloscore.iterate_elo_score_calculation_for_dataframe(dataframe=cage_df, winner_column="winner", loser_column="loser", additional_columns=["index", "date", "cage", "box", "match", "scorer", "animal_ids", "trial", "trial_number"])

In [ ]:
cage_to_elo_score_dict.keys()

In [ ]:
cage_to_elo_score_dict[list(cage_to_elo_score_dict.keys())[0]][0]

- Turning the dictionary into a dataframe

In [ ]:
all_cage_elo_score_list = []
for key in cage_to_elo_score_dict.keys():
    cage_elo_score_df = pd.DataFrame.from_dict(cage_to_elo_score_dict[key], orient="index")
    cage_elo_score_df.insert(0, 'total_trial_number', range(0, 0 + len(cage_elo_score_df)))
    print(cage_elo_score_df)
    all_cage_elo_score_list.append(cage_elo_score_df)


In [ ]:
all_cage_elo_score_df = pd.concat(all_cage_elo_score_list)

In [ ]:
all_cage_elo_score_df.head()

In [ ]:
all_cage_elo_score_df.tail()

In [ ]:
if cage_to_strain:
    all_cage_elo_score_df["strain"] = all_cage_elo_score_df["cage"].astype(str).map(cage_to_strain)

- Adding the cohort

In [ ]:
all_cage_elo_score_df["experiment_type"] = protocol_name
all_cage_elo_score_df["cohort"] = cohort_name

In [ ]:
all_cage_elo_score_df.head()

In [ ]:
all_cage_elo_score_df.head()

## Saving the Elo Score Dataframes

- Create folders to save processed data

- Adding the cage information for each cage

In [ ]:
all_cage_elo_score_df.head()

In [ ]:
id_to_final_elo_score_dict = defaultdict(dict)
for index, subject_id in enumerate(sorted(all_cage_elo_score_df["subject_id"].unique())):
    per_subject_df = all_cage_elo_score_df[all_cage_elo_score_df["subject_id"] == subject_id]
    id_to_final_elo_score_dict[index]["subject_id"] = subject_id

    id_to_final_elo_score_dict[index]["final_elo_score"] = per_subject_df.iloc[-1]["updated_elo_score"]
    id_to_final_elo_score_dict[index]["cohort"] = per_subject_df.iloc[-1]["cohort"]
    id_to_final_elo_score_dict[index]["strain"] = per_subject_df.iloc[-1]["strain"]
    id_to_final_elo_score_dict[index]["cage"] = per_subject_df.iloc[-1]["cage"]

    

In [ ]:
id_to_final_elo_score_dict

In [ ]:
id_to_final_elo_score_df = pd.DataFrame.from_dict(id_to_final_elo_score_dict, orient="index")
# Adding protocol name
id_to_final_elo_score_df["experiment_type"] = protocol_name
# Adding rank
id_to_final_elo_score_df["rank"] = id_to_final_elo_score_df.groupby("cage")["final_elo_score"].rank("dense", ascending=False)
# Sorting by cage and then id
id_to_final_elo_score_df = id_to_final_elo_score_df.sort_values(by=['cage', "subject_id"], ascending=True).reset_index(drop=True)

In [ ]:
id_to_final_elo_score_df["rank"] = id_to_final_elo_score_df.groupby("cage")["final_elo_score"].rank("dense", ascending=False)

In [ ]:
id_to_final_elo_score_df = id_to_final_elo_score_df.sort_values(by=['cage', "subject_id"], ascending=True).reset_index(drop=True)

In [ ]:
id_to_final_elo_score_df.head()

In [ ]:
id_to_final_elo_score_df.tail()

# Plotting the Elo Score by match number

In [ ]:
plot_output_directory = "./proc/plots/reward_competition/cage_{}_date_{}_{}".format(all_cages, earliest_date, latest_date)

In [ ]:
plot_output_directory

In [ ]:
os.makedirs(plot_output_directory, exist_ok=True)

In [ ]:
all_cage_elo_score_df["subject_id"] = all_cage_elo_score_df["subject_id"].astype(str)
all_cage_elo_score_df["agent_id"] = all_cage_elo_score_df["agent_id"].astype(str)

In [ ]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

In [ ]:
for cage in all_cage_elo_score_df["cage"].unique():
    fig, ax = plt.subplots()
    plt.rcParams["figure.figsize"] = (18,10)
    per_cage_df = all_cage_elo_score_df[all_cage_elo_score_df["cage"] == cage]
       
    for index in per_cage_df["index"].unique():
        first_session_in_trial = per_cage_df[per_cage_df["index"] == index].iloc[0]["total_trial_number"]
        plt.vlines(x=[first_session_in_trial - 0.5], ymin=700, ymax=1300, colors='black', linestyle='dashed')

    
    
    # Drawing a line for each subject
    for subject in sorted(per_cage_df["subject_id"].unique()):
        # Getting all the rows with the current subject
        subject_df = per_cage_df[per_cage_df["subject_id"] == subject]
        # Making the dates into days after the first session by subtracting all the dates by the first date
        plt.plot(subject_df["total_trial_number"], subject_df["updated_elo_score"], '-o', label=subject)
    
    # Labeling the X/Y Axis and the title
    ax.set_xlabel("Trial Number")
    ax.set_ylabel("Elo Score")
    ax.set_title("{} Elo Score for {}: Cage {}".format(" ".join(cohort_name.split("_")).capitalize(), string.capwords(" ".join(protocol_name.split("_"))), cage))
    # To show the legend
    ax.legend(loc="upper left")
    plt.xticks(rotation=90)
    plt.ylim(700, 1300) 
    file_name = "{}_{}_elo_score_{}_date_{}_{}.png".format(cohort_name, protocol_name, cage, earliest_date, latest_date)
    plt.savefig(os.path.join(plot_output_directory, file_name))  
    
    plt.show()

# Saving the Dataframes

In [ ]:
elo_score_spreadsheet_output_directory = "./proc/elo_score_spread_sheets/{}_elo_score/cage_{}_date_{}_{}".format(protocol_name, all_cages, earliest_date, latest_date)

In [ ]:
elo_score_spreadsheet_output_directory

In [ ]:
os.makedirs(elo_score_spreadsheet_output_directory, exist_ok=True)

In [ ]:
file_name = "{}_elo_score_history_cages_{}_date_{}_{}.csv".format(protocol_name, all_cages, earliest_date, latest_date)
all_cage_elo_score_df.to_csv(os.path.join(elo_score_spreadsheet_output_directory, file_name))

In [ ]:
file_name = "{}_final_elo_score_cages_{}_date_{}_{}.csv".format(protocol_name, all_cages, earliest_date, latest_date)
id_to_final_elo_score_df.to_csv(os.path.join(elo_score_spreadsheet_output_directory, file_name))

## Seeing which subject is the dominant or submissive

- Grouping all the rows with the same pair

In [ ]:
all_processed_behavior_recording_df = melted_reward_competition_df

In [ ]:
all_processed_behavior_recording_df.head()

In [ ]:
all_processed_behavior_recording_df.columns

In [ ]:
['date', 'cage', 'match', 'animal_ids', 'trial', 'winner', 'tie', 'loser']

- Removing columns that would be unnecessary for the pairings

In [ ]:
all_processed_behavior_recording_df = all_processed_behavior_recording_df[['date', 'cage', 'match', 'animal_ids', 'trial', 'winner', 'tie', 'loser']]
all_processed_behavior_recording_df['processed_cage_number'] = all_processed_behavior_recording_df['cage'].astype(int)

In [ ]:
all_processed_behavior_recording_df.head()

- Adding a tie to the list of winners and losers if it's a tie

In [ ]:
all_processed_behavior_recording_df["winner"] = all_processed_behavior_recording_df.apply(lambda x: "tie" if x["tie"] else x["winner"], axis=1)
all_processed_behavior_recording_df["loser"] = all_processed_behavior_recording_df.apply(lambda x: "tie" if x["tie"] else x["loser"], axis=1)

In [ ]:
all_processed_behavior_recording_df[all_processed_behavior_recording_df["tie"]].head()

- Making a list of all the wins and loses

In [ ]:
all_wins_per_pair = all_processed_behavior_recording_df.groupby(["animal_ids", "date"])['winner'].apply(list)
all_loses_per_pair = all_processed_behavior_recording_df.groupby(["animal_ids", "date"])['loser'].apply(list)

In [ ]:
all_competition_per_pair_df = pd.concat([all_wins_per_pair, all_loses_per_pair], axis=1).reset_index()

In [ ]:
all_competition_per_pair_df = all_competition_per_pair_df.rename(columns={k: "rc_" + k for k in all_competition_per_pair_df.columns})

In [ ]:
all_competition_per_pair_df

- Seeing how often the winners or losers change

In [ ]:
# Dropping all the ties
all_competition_per_pair_df["rc_winner_no_ties"] = all_competition_per_pair_df.apply(lambda row: [x for x in row["rc_winner"] if x in row["rc_animal_ids"]], axis=1)
all_competition_per_pair_df["rc_loser_no_ties"] = all_competition_per_pair_df.apply(lambda row: [x for x in row["rc_loser"] if x in row["rc_animal_ids"]], axis=1)

In [ ]:
# Comparing the each item with the item after it to see if they are the same of not
all_competition_per_pair_df["rc_different_result_as_previous"] = all_competition_per_pair_df["rc_winner_no_ties"].apply(lambda x: [True if left != right else False for (left, right) in zip(x[1:], x[:-1])])

In [ ]:
all_competition_per_pair_df["rc_number_of_switches"] = all_competition_per_pair_df["rc_different_result_as_previous"].apply(lambda x: sum(x))

In [ ]:
all_competition_per_pair_df

# NOTE: This assumes that Ties aren't the most common outcome for a given pair

## Aggregate all the wins/loses across reward competition sessions

- Getting the average number of switches between winner and loser

In [ ]:
# Renaming column to not be confused with previous version
average_switches_per_pair_df = all_competition_per_pair_df.groupby('rc_animal_ids').mean().rename({'rc_number_of_switches': 'rc_average_number_of_switches'}, axis=1)  # new method

- Combining all the trials into one list

In [ ]:
all_sessions_per_pair_df = pd.concat([all_competition_per_pair_df.groupby('rc_animal_ids')['rc_winner'].apply(list), all_competition_per_pair_df.groupby('rc_animal_ids')['rc_loser'].apply(list)], axis=1)
all_sessions_per_pair_df = pd.concat([all_sessions_per_pair_df, average_switches_per_pair_df], axis=1)
all_sessions_per_pair_df = all_sessions_per_pair_df.reset_index()

In [ ]:
all_sessions_per_pair_df.head()

- Flattening the list

In [ ]:
all_sessions_per_pair_df["rc_winner"] = all_sessions_per_pair_df["rc_winner"].apply(lambda x: [element for innerList in x for element in innerList])
all_sessions_per_pair_df["rc_loser"] = all_sessions_per_pair_df["rc_loser"].apply(lambda x: [element for innerList in x for element in innerList])

In [ ]:
all_sessions_per_pair_df.head()

- Calculating overall winner and loser based on who has the most wins/loses

In [ ]:
# Dropping all the ties
all_sessions_per_pair_df["rc_winner_no_ties"] = all_sessions_per_pair_df.apply(lambda row: [x for x in row["rc_winner"] if x in row["rc_animal_ids"]], axis=1)
all_sessions_per_pair_df["rc_loser_no_ties"] = all_sessions_per_pair_df.apply(lambda row: [x for x in row["rc_loser"] if x in row["rc_animal_ids"]], axis=1)

In [ ]:
# Putting the first ID for the winner and second for the loser if there is equal number of wins and loses 
all_sessions_per_pair_df["rc_averaged_winner"] = all_sessions_per_pair_df.apply(lambda x: Counter(x["rc_winner_no_ties"]).most_common(1)[0][0] if x["rc_winner_no_ties"].count(Counter(x["rc_winner_no_ties"]).most_common(1)[0][1]) != len(x["rc_winner_no_ties"]) / 2 else x["rc_animal_ids"][0], axis=1)
all_sessions_per_pair_df["rc_averaged_loser"] = all_sessions_per_pair_df.apply(lambda x: Counter(x["rc_loser_no_ties"]).most_common(1)[0][0] if x["rc_loser_no_ties"].count(Counter(x["rc_loser_no_ties"]).most_common(1)[0][1]) != len(x["rc_winner_no_ties"]) / 2 else x["rc_animal_ids"][1], axis=1)


- Checking to see if any of the IDs are the same or are ties

In [ ]:
all_sessions_per_pair_df["rc_averaged_loser"].unique()

In [ ]:
all_sessions_per_pair_df["rc_averaged_winner"].unique()

In [ ]:
all_sessions_per_pair_df[all_sessions_per_pair_df["rc_averaged_loser"] == all_sessions_per_pair_df["rc_averaged_winner"]]

- Counting the number of wins and loses

In [ ]:
all_sessions_per_pair_df["rc_winner_count"] = all_sessions_per_pair_df.apply(lambda x: x["rc_winner"].count(x["rc_averaged_winner"]), axis=1)
all_sessions_per_pair_df["rc_loser_count"] = all_sessions_per_pair_df.apply(lambda x: x["rc_winner"].count(x["rc_averaged_loser"]), axis=1)
all_sessions_per_pair_df["rc_tie_count"] = all_sessions_per_pair_df.apply(lambda x: x["rc_winner"].count("tie"), axis=1)
all_sessions_per_pair_df["rc_all_match_count_including_ties"] = all_sessions_per_pair_df["rc_winner"].apply(lambda x: len(x))

- Comparing the number of wins vs loses

In [ ]:
all_sessions_per_pair_df["rc_winner_count_minus_loser_count"] = all_sessions_per_pair_df["rc_winner_count"] - all_sessions_per_pair_df["rc_loser_count"]
# winner number / (winner + loser number with no ties)
all_sessions_per_pair_df["rc_win_to_win_plus_lost_ratio"] = all_sessions_per_pair_df.apply(lambda x: x["rc_winner_count"] / len(x["rc_winner_no_ties"]), axis=1)
# winner number / (winner + loser + tie)
all_sessions_per_pair_df["rc_win_to_all_ratio"] = all_sessions_per_pair_df.apply(lambda x: x["rc_winner_count"] / x["rc_all_match_count_including_ties"], axis=1)
# all_sessions_per_pair_df["rc_is_win_to_all_ratio_tie"] = all_sessions_per_pair_df["rc_win_to_all_ratio"].apply(lambda x: True if x < 0.5 else False)
all_sessions_per_pair_df["rc_is_win_to_win_and_loss_ratio_tie"] = all_sessions_per_pair_df["rc_win_to_win_plus_lost_ratio"].apply(lambda x: True if x < 0.6 else False)

In [ ]:
all_sessions_per_pair_df["rc_is_win_to_win_and_loss_ratio_tie"]

- Comparing the number of ties to the number of matches

In [ ]:
all_sessions_per_pair_df["rc_tie_to_all_ratio"] = all_sessions_per_pair_df.apply(lambda x: x["rc_tie_count"] / x["rc_all_match_count_including_ties"], axis=1)


In [ ]:
all_sessions_per_pair_df[~all_sessions_per_pair_df["rc_is_win_to_win_and_loss_ratio_tie"]]

In [ ]:
all_sessions_per_pair_df.head()

- Checking to see if there are any pairs with more than two sessions

In [ ]:
all_sessions_per_pair_df[all_sessions_per_pair_df["rc_all_match_count_including_ties"] > 40]

- Adding the cage information

In [ ]:
dropped_cuplicate_all_processed_behavior_recording_df = all_processed_behavior_recording_df[["animal_ids", "processed_cage_number"]].drop_duplicates()

In [ ]:
dropped_cuplicate_all_processed_behavior_recording_df

In [ ]:
pair_to_cage = pd.Series(dropped_cuplicate_all_processed_behavior_recording_df["processed_cage_number"].values, index=dropped_cuplicate_all_processed_behavior_recording_df["animal_ids"]).to_dict()

In [ ]:
pair_to_cage

In [ ]:
all_sessions_per_pair_df["processed_cage_number"] = all_sessions_per_pair_df["rc_animal_ids"].map(pair_to_cage)

In [ ]:
all_sessions_per_pair_df

In [ ]:
all_sessions_per_pair_df["cohort"] = cohort_name
all_sessions_per_pair_df["processed_cage_number"] = all_sessions_per_pair_df["processed_cage_number"].astype(int).astype(str)

In [ ]:
all_sessions_per_pair_df.head()

- Verifying if all the stats are correct

In [ ]:
all_sessions_per_pair_df["rc_winner"][0].count(all_sessions_per_pair_df["rc_animal_ids"][0][0])

In [ ]:
all_sessions_per_pair_df["rc_winner"][0].count(all_sessions_per_pair_df["rc_animal_ids"][1][0])

In [ ]:
all_sessions_per_pair_df["rc_winner"][0].count("tie")

In [ ]:
len(all_sessions_per_pair_df["rc_winner"][0])

In [ ]:
len(all_sessions_per_pair_df["rc_winner_no_ties"][0])

In [ ]:
all_sessions_per_pair_df.tail()

# Getting the number of wins per winner and loser

In [ ]:
all_competition_per_pair_df["cohort"] = cohort_name

In [ ]:
all_competition_per_pair_df["processed_cage_number"] = all_competition_per_pair_df["rc_animal_ids"].map(pair_to_cage).astype(str)

In [ ]:
all_competition_per_pair_df["strain"] = all_competition_per_pair_df["processed_cage_number"].map(cage_to_strain)

In [ ]:
all_competition_per_pair_df.head()

- Calculating overall winner and loser based on who has the most wins/loses

In [ ]:
# Dropping all the ties
all_competition_per_pair_df["rc_winner_no_ties"] = all_competition_per_pair_df.apply(lambda row: [x for x in row["rc_winner"] if x in row["rc_animal_ids"]], axis=1)
all_competition_per_pair_df["rc_loser_no_ties"] = all_competition_per_pair_df.apply(lambda row: [x for x in row["rc_loser"] if x in row["rc_animal_ids"]], axis=1)

In [ ]:
# Putting the first ID for the winner and second for the loser if there is equal number of wins and loses 
all_competition_per_pair_df["rc_averaged_winner"] = all_competition_per_pair_df.apply(lambda x: Counter(x["rc_winner_no_ties"]).most_common(1)[0][0] if x["rc_winner_no_ties"].count(Counter(x["rc_winner_no_ties"]).most_common(1)[0][1]) != len(x["rc_winner_no_ties"]) / 2 else x["rc_animal_ids"][0], axis=1)
all_competition_per_pair_df["rc_averaged_loser"] = all_competition_per_pair_df.apply(lambda x: Counter(x["rc_loser_no_ties"]).most_common(1)[0][0] if x["rc_loser_no_ties"].count(Counter(x["rc_loser_no_ties"]).most_common(1)[0][1]) != len(x["rc_winner_no_ties"]) / 2 else x["rc_animal_ids"][1], axis=1)


In [ ]:
Counter(all_competition_per_pair_df["rc_winner_no_ties"][0]).most_common(1)[0][1]

- Checking to see if any of the IDs are the same or are ties

In [ ]:
all_competition_per_pair_df["rc_averaged_loser"].unique()

In [ ]:
all_competition_per_pair_df["rc_averaged_winner"].unique()

- Counting the number of wins and loses

In [ ]:
all_competition_per_pair_df["rc_winner_count"] = all_competition_per_pair_df.apply(lambda x: x["rc_winner"].count(x["rc_averaged_winner"]), axis=1)
all_competition_per_pair_df["rc_loser_count"] = all_competition_per_pair_df.apply(lambda x: x["rc_winner"].count(x["rc_averaged_loser"]), axis=1)
all_competition_per_pair_df["rc_tie_count"] = all_competition_per_pair_df.apply(lambda x: x["rc_winner"].count("tie"), axis=1)
all_competition_per_pair_df["rc_all_match_count_including_ties"] = all_competition_per_pair_df["rc_winner"].apply(lambda x: len(x))

In [ ]:
all_competition_per_pair_df.head()

- Comparing the number of wins vs loses

In [ ]:
all_competition_per_pair_df["rc_winner_count_minus_loser_count"] = all_competition_per_pair_df["rc_winner_count"] - all_competition_per_pair_df["rc_loser_count"]
# winner number / (winner + loser number with no ties)
all_competition_per_pair_df["rc_win_to_win_plus_lost_ratio"] = all_competition_per_pair_df.apply(lambda x: x["rc_winner_count"] / len(x["rc_winner_no_ties"]), axis=1)
# winner number / (winner + loser + tie)
all_competition_per_pair_df["rc_win_to_all_ratio"] = all_competition_per_pair_df.apply(lambda x: x["rc_winner_count"] / x["rc_all_match_count_including_ties"], axis=1)
all_competition_per_pair_df["rc_is_win_to_win_and_loss_ratio_tie"] = all_competition_per_pair_df["rc_win_to_win_plus_lost_ratio"].apply(lambda x: True if x < 0.6 else False)

In [ ]:
all_competition_per_pair_df["rc_is_win_to_win_and_loss_ratio_tie"]

- Comparing the number of ties to the number of matches

In [ ]:
all_competition_per_pair_df["rc_tie_to_all_ratio"] = all_competition_per_pair_df.apply(lambda x: x["rc_tie_count"] / x["rc_all_match_count_including_ties"], axis=1)


- Saving the competition dataframe

In [ ]:
file_name = "{}_{}_grouped_by_pairs_cage_{}_date_{}_{}.csv".format(cohort_name, protocol_name, all_cages, earliest_date, latest_date)


In [ ]:
elo_score_spreadsheet_output_directory

In [ ]:
file_name

In [ ]:
file_name = "{}_{}_all_competition_cage_{}_date_{}_{}.csv".format(cohort_name, protocol_name, all_cages, earliest_date, latest_date)


In [ ]:
elo_score_spreadsheet_output_directory

In [ ]:
file_name

In [ ]:
all_competition_per_pair_df.to_csv(os.path.join(elo_score_spreadsheet_output_directory, file_name))

# Saving the spreadsheets

In [ ]:
file_name = "{}_{}_grouped_by_pairs_cage_{}_date_{}_{}.csv".format(cohort_name, protocol_name, all_cages, earliest_date, latest_date)


In [ ]:
elo_score_spreadsheet_output_directory

In [ ]:
file_name

In [ ]:
all_sessions_per_pair_df.to_csv(os.path.join(elo_score_spreadsheet_output_directory, file_name))

In [ ]:
reward_competition_df